# Import Dependencies 

In [ ]:
from math import gcd,sqrt
from tqdm import tqdm
from math import floor
import math
import itertools
import pandas as pd
import plotly.express as px
from sympy.combinatorics import Permutation

# Prequisite Functions

In [ ]:
def coprime(a, b):
    return gcd(int(a), int(b)) == 1

def factors_of_n(n):
    factor_list=[]
    i=1
    while i<=n:
        if (n%i==0):
            factor_list.append(i)
        i=i+1
    return factor_list

def factorial(n):
    fact = 1
    for i in range(1,n+1):
        fact = fact*i
    return fact

def totient(n):
    amount = 0        
    for k in range(1, n + 1):
        if gcd(n, k) == 1:
            amount += 1
    return amount

def new_power(x,y):
    if x==0 and y==0:
        return 1
    else:
        return x**y

def get_inverse(perm,n):
    perm_inverse = ()
    for i in range(1, n):
        perm_inverse += (perm.index(i)+1,)
    perm_inverse+= (n,)
    return perm_inverse

# Direct Computation

In [ ]:
def get_j_values(n,t):
    j_values = []
    if t==n:
        return [n]
    for integer in range(int(n/t)):
        if coprime(integer,n/t):
            j_values.append(integer*t)
    return j_values

def is_unique_list(element_list):
    return len(element_list) == len(set(element_list))

def count_fixed_points(perm,n):
    total_fixed = 0
    for i in range(0,n):
        if perm[i] == i+1:
            total_fixed +=1
    return total_fixed

def get_perm_list(n,t):
    index_set = []
    j_values = get_j_values(n,t)
    index_t_set = j_values
    for i in range(1,t):
        index_i_set = [num for num in range(1,n) if num not in j_values]
        index_set.append(index_i_set)
    index_set.append(index_t_set)
    cardinality = 1
    for element_index in index_set:
        cardinality *= len(element_index)
        if cardinality>1e8:
            return[]
    perm_list = []
    for perm in itertools.product(*index_set):
        perm_list.append(perm)
    return perm_list

def filter_perms(perm_list,t):
    filtered_list = []
    for perm in perm_list:
        if is_unique_list([element%t for element in perm]):
            filtered_list.append(perm)
    return filtered_list

def extend_list(filtered_list,n,t):
    extended_list = []
    for perm in filtered_list:
        if t==n:
            return filtered_list
        for element_index in range(t+1,n):
            multiplier = floor(element_index/t)
            remainder = element_index%t
            if remainder==0:
                element = int(multiplier*perm[t-1])%n
            else:
                element = int(perm[remainder-1]+multiplier*perm[t-1])%n
            perm += (element,)
        perm+=(n,)
        extended_list.append(perm)
    return extended_list

def generate_full_perm_list(n,t):
    perm_list = get_perm_list(n,t)
    if perm_list == []:
        return []
    filtered_list = filter_perms(perm_list,t)
    return extend_list(filtered_list,n,t)

def remove_overcount(n,t):
    factors = factors_of_n(t)[:-1]
    full_perm_list_given_t = generate_full_perm_list(n,t)
    if full_perm_list_given_t == []:
        return []
    for factor in factors:
        full_perm_list_given_t = set(full_perm_list_given_t).difference(set(generate_full_perm_list(n,factor)))
    return full_perm_list_given_t

def list_involutions(n,t):
    all_perms = remove_overcount(n,t)
    involutions_list = []
    for perm in all_perms:
        if get_inverse(perm,n)==perm:
            involutions_list.append(perm)
    return involutions_list

def list_permutations_with_r_fixed_points(n,t,r):
    inv_with_fixed_list = list()
    all_perms = remove_overcount(n,t)
    for perm in all_perms:
        fixed_pts = count_fixed_points(perm,n)
        if fixed_pts == r:
            inv_with_fixed_list.append(perm)
    return inv_with_fixed_list

def list_involutions_with_r_fixed_points(n,t,r):
    if (n-r)%2==1:
        return list()
    else:
        inv_with_fixed_list = list()
        all_inv = list_involutions(n,t)
        for perm in all_inv:
            fixed_pts = count_fixed_points(perm,n)
            if fixed_pts == r:
                inv_with_fixed_list.append(perm)
        return inv_with_fixed_list

def list_invlutions_with_i_fixed_pts(n,t,i):
    all_inv = list_involutions(n,t)
    inv_list = []
    for perm in all_inv:
        inv_total = []
        for j in range(0,n):
            if perm[j]==j+1:
                inv_total.append(j)
        if len(inv_total)==i:
            inv_list.append(perm)
    return inv_list

def list_invlutions_with_orbits_containing_i_involutions(n,t,i):
    all_inv = list_involutions(n,t)
    inv_list = []
    for perm in all_inv:
        orbit_of = orbit(perm,n)
        inv_total = []
        for perm_2 in orbit_of:
            if get_inverse(perm_2,n)==perm_2:
                inv_total.append(perm_2)
        if len(inv_total)==i:
            inv_list.append(perm)
    return inv_list

def orbit(perm,n):
    orbit = set()
    orbit.add(perm)
    for i in range(1,n):
        perm_i = ()
        for j in range(1,n):
            i_plus_j_mod_n = (i+j)%n
            perm_i_index_j = (perm[i_plus_j_mod_n-1]-perm[i-1])%n
            perm_i+= (perm_i_index_j,)
        perm_i+=(n,)
        orbit.add(perm_i)
    return list(orbit)

def get_orbits(n,t):
    all_perms_with_stab_gen_by_t = remove_overcount(n,t)
    if all_perms_with_stab_gen_by_t == []:
        return []
    orbit_set = []
    for perm in all_perms_with_stab_gen_by_t:
        orbit_of_perm = orbit(perm,n)
        if len(orbit_of_perm)==t:
            orbit_set.append(orbit_of_perm)
    return orbit_set

def get_unique_orbits(n,t):
    all_orbits = get_orbits(n,t)
    if all_orbits == []:
        return []
    distinct_orbits = set()
    distinct_orbits_list = []
    for orbit in all_orbits:
        distinct_orbits.add(frozenset(orbit))
    for orbit in distinct_orbits:
        orb = list(orbit)
        distinct_orbits_list.append(orb)
    return distinct_orbits_list

def get_orbits_with_involutions(n,t):
    all_orbits = get_unique_orbits(n,t)
    total = set()
    for orbit in all_orbits:
        for perm in orbit:
            if get_inverse(perm,n)==perm:
                total.add(frozenset(orbit))
    return list(total)

def list_orbits_with_involutions(n,t):
    all_orbits = get_unique_orbits(n,t)
    total = []
    for orbit in all_orbits:
        involutions = []
        for perm in orbit:
            if get_inverse(perm,n)==perm:
                involutions.append(perm)
        total.append(len(involutions))
    return total

def count_orbits_with_i_involutions(n,t,i):
    list_total_orbits_with_involutions = list_orbits_with_involutions(n,t)
    count = 0
    for ele in list_total_orbits_with_involutions:
        if ele==i:
            count+=1
    return count

def list_orbits_with_i_involutions(n,t,i):
    all_orbits = get_unique_orbits(n,t)
    total = list()
    for orbit in all_orbits:
        involution = 0
        for perm in orbit:
            if get_inverse(perm,n)==perm:
#                 perm_new = list(perm)
#                 perm_new.insert(0,0)
#                 print(f"perm = {Permutation(perm_new)}")
                involution +=1
#         print(f"number of involutions = {involution}")
        if involution==i:
#             print(f"{involution} involutions in {orbit}")
            total.append(orbit)
    return total

def count_orbits_with_i_involutions_all_i(n,t):
    list_all_occurences = []
    for i in range(0,t+1):
        if (t-i)%2==0:
            num_of_occurences = count_orbits_with_i_involutions(n,t,i)
            list_all_occurences.append([f"{num_of_occurences} orbits with {i} involutions"])
    return list_all_occurences

def get_inverse_s_value(perm,n):
    inverse = get_inverse(perm,n)
    orbit_of_perm = orbit(perm,n)
    t = len(orbit_of_perm)
    if inverse not in orbit_of_perm:
        return print(f"inverse not in orbit; no s")
    if inverse == perm:
        return 0
    else:
        for i in range(1,t):
            perm_i = ()
            for j in range(1,n):
                i_plus_j_mod_n = (i+j)%n
                perm_i_index_j = (perm[i_plus_j_mod_n-1]-perm[i-1])%n
                perm_i+= (perm_i_index_j,)
            perm_i+=(n,)
            if perm_i==inverse:
                return i
            

def get_orbits_with_inverse_in_orbit(n,t):
    all_orbits = get_unique_orbits(n,t)
    if get_unique_orbits == []:
        return []
    all_orbits_with_inverses = []
    for orbit_guy in all_orbits:
        element_of = orbit_guy[0]
        inverse_element_of = get_inverse(element_of,n)
        if inverse_element_of in orbit_guy:
            all_orbits_with_inverses.append(orbit_guy)
    return all_orbits_with_inverses

def print_indicators_for_permutation_given_n_and_perm(n,perm):
    perm_orbit = orbit(perm,n)
    t = len(perm_orbit)
    nt = int(n/t)
    inverse = get_inverse(perm,n)
    if inverse not in perm_orbit:
        return print(f"There are {nt} irrep(s) of dimension {t} and each has indicator 0")
    s_value = get_inverse_s_value(perm,n)
    if nt%2==1:
        k1_raw = (perm[t-1]+t)%n
        k1 = int(k1_raw/t)
        return print(f"Indicator is nonnegative. There are {nt} irrep(s) of dimension {t}; {gcd(k1,nt)} with indicator +1 and {nt-gcd(k1,nt)} with indicator 0")
    else:
        k1_raw = (perm[t-1]+t)%n
        k1 = int(k1_raw/t)
        if inverse==perm:
            return print(f"Indicator is nonnegative. There are {nt} irrep(s) of dimension {t}; {gcd(k1,nt)} with indicator +1 and {nt-gcd(k1,nt)} with indicator 0")
        else: 
            k2_raw = (perm[s_value-1]+s_value)%n
            k2 = int(k2_raw/t)
        list_of_products = list()
        for i in range(0,nt):
            iszero = i*k1 %nt
            if iszero == 0:
                product = (-i*k2) % nt
                list_of_products.append(product)
        plus_ones = list_of_products.count(0)
        minus_ones = len(list_of_products) - plus_ones
        zeros = nt-len(list_of_products)
        return print(f"There are {nt} irrep(s) of dimension {t}; {plus_ones} with indicator +1, {minus_ones} with indicator -1, and {zeros} with indicator 0")

def list_indicators_for_permutation_given_n_and_perm_plus_minus_zero(n,perm):
    perm_orbit = orbit(perm,n)
    t = len(perm_orbit)
    nt = int(n/t)
    inverse = get_inverse(perm,n)
    if inverse not in perm_orbit:
        return [0,0,nt]
    s_value = get_inverse_s_value(perm,n)
    if nt%2==1:
        k1_raw = (perm[t-1]+t)%n
        k1 = int(k1_raw/t)
        return [gcd(k1,nt),0,nt-gcd(k1,nt)]
    else:
        k1_raw = (perm[t-1]+t)%n
        k1 = int(k1_raw/t)
        if inverse==perm:
            return [gcd(k1,nt),0,nt-gcd(k1,nt)]
        else: 
            k2_raw = (perm[s_value-1]+s_value)%n
            k2 = int(k2_raw/t)
        list_of_products = list()
        for i in range(0,nt):
            iszero = i*k1 %nt
            if iszero == 0:
                product = (-i*k2) % nt
                list_of_products.append(product)
        plus_ones = list_of_products.count(0)
        minus_ones = len(list_of_products) - plus_ones
        zeros = nt-len(list_of_products)
        return [plus_ones,minus_ones,zeros]
    
def total_indicators_for_irreps_dim_t_given_n(n,t):
    unique_orbits = get_unique_orbits(n,t)
    total_plus_ones_list = list()
    total_minus_ones_list = list()
    total_zeros_list = list()
    for orbit in unique_orbits:
        perm = orbit[0]
        list_of_indicators = list_indicators_for_permutation_given_n_and_perm_plus_minus_zero(n,perm)
        plus_ones = list_of_indicators[0]*t
        minus_ones = list_of_indicators[1]*t
        zeros = list_of_indicators[2]*t
        total_plus_ones_list.append(plus_ones)
        total_minus_ones_list.append(minus_ones)
        total_zeros_list.append(zeros)
    total_plus_ones = sum(total_plus_ones_list)
    total_minus_ones = sum(total_minus_ones_list)
    total_zeros = sum(total_zeros_list)
    return print(f"Total irreps of dimension {t} with: \n indicator +1: {total_plus_ones} \n indicator -1: {total_minus_ones} \n indicator 0: {total_zeros}")

def total_indicators_for_irreps_dim_t_given_n_as_list_plus_minus_zero(n,t):
    unique_orbits = get_unique_orbits(n,t)
    total_plus_ones_list = list()
    total_minus_ones_list = list()
    total_zeros_list = list()
    for orbit in unique_orbits:
        perm = orbit[0]
        list_of_indicators = list_indicators_for_permutation_given_n_and_perm_plus_minus_zero(n,perm)
        plus_ones = list_of_indicators[0]*t
        minus_ones = list_of_indicators[1]*t
        zeros = list_of_indicators[2]*t
        total_plus_ones_list.append(plus_ones)
        total_minus_ones_list.append(minus_ones)
        total_zeros_list.append(zeros)
    total_plus_ones = sum(total_plus_ones_list)
    total_minus_ones = sum(total_minus_ones_list)
    total_zeros = sum(total_zeros_list)
    return [total_plus_ones,total_minus_ones,total_zeros]

# My Formulas

In [ ]:
def E_n_over_t_list(n,t):
    if t==n:
        return [1]
    else:
        E_list = list()
        nt = int(n/t)
        for j in range(0,nt):
            new_j_square = (j*j % nt)
            if new_j_square == 1:
                E_list.append(j)
        return E_list

def K_tj_list(n,t,j):
    if t==n:
        return [1]
    else:
        k_values = list()
        for k in range(0,int(n/t)):
            if (k*(j+1))%int(n/t)==0:
                k_values.append(k)
    return k_values

def Special_E_from_sigma(n,t,s,j):
    Special_E_from_sigma_list = set()
    E_list = E_n_over_t_list(n,s)
    nt = int(n/t)
    ns = int(n/s)
    for m in range(0,nt):
        value = j+m*t/s
        int_value = int(value)
        int_value_mod = int_value % ns
        if int_value_mod in E_list:
            Special_E_from_sigma_list.add(int_value_mod)
    return list(Special_E_from_sigma_list)

def alpha_value(n,t,j):
    nt = int(n/t)
    return gcd(j+1,nt)

def beta_value(n,t,j):
    nt = int(n/t)
    return gcd(j-1,nt)

def delta_zero_if(n,t,r,j):
    nt = int(n/t)
    get_gcd = beta_value(n,t,j)
    if r % get_gcd == 0:
        if r<=t*get_gcd:
            return 1
    else:
        return 0
    
def delta_zero_if_j(n,t,j_prime,j):
    value1 = j_prime*t % n
    value2 = j*t % n
    if value1==value2:
        return 1
    else:
        return 0
    
def P_list(n,t,j):
    P_list = set()
    nt = int(n/t)
    #print(f"n={n},t={t},n/t={nt}")
    for q in range(0,nt):
        value = int(q*(1-j))
        value_mod = (value % nt)
        #print(f"P_list_value = {value_mod}")
        P_list.add(value_mod)
    return list(P_list)

def P_list_compliment(n,t,j):
    P_comp_list = []
    nt = int(n/t)
    K_set = K_tj_list(n,t,j)
    P_compliment = [i for i in list(range(0,nt)) if i not in P_list(n,t,j)]
    for i in range(0,nt):
        if i in P_compliment:
            if i in K_set:
                P_comp_list.append(i)
    return P_comp_list

def K_P_compliment_intersection_set(n,t,j):
    K_set = K_tj_list(n,t,j)
    P_compliment_set = P_list_compliment(n,t,j)
    new_set = set()
    for k in K_set:
        if k in P_compliment_set:
            new_set.add(k)
    return list(new_set)
    
def Special_K(n,t,s,j_sigma,j_prime):
    Special_K_list = list()
    ns = int(n/s)
    nt = int(n/t)
    ts = int(t/s)
    P_list_set = P_list(t,s,j_sigma)
    for m in P_list_set:
        for k in range(0,nt):
            value = (k*ts+m)*(1+j_prime)
            value_int = int(value)
            if (value_int % ns) == 0:
                Special_K_list.append([k,m])
    return list(Special_K_list)

def Special_K_compliment(n,t,s,j_sigma,j_prime):
    Special_K_compliment_list = list()
    ns = int(n/s)
    nt = int(n/t)
    ts = int(t/s)
    PK_list_compliment_set = K_P_compliment_intersection_set(t,s,j_sigma)
    for m in PK_list_compliment_set:
        for k in range(0,nt):
            value = (k*ts+m)*(1+j_prime)
            value_int = int(value)
            if (value_int % ns) == 0:
                Special_K_compliment_list.append([k,m])
    return list(Special_K_compliment_list)

def delta_zero_for_P_compliment_set(n,t,j,r,l):
    PKComp = len(K_P_compliment_intersection_set(n,t,j))
    if PKComp==0:
        #print(f"PKComp = {PKComp}")
        beta = beta_value(n,t,j)
        m = int(r/beta)
        if 2*l!=t-m:
            #print(f"2l = {2*l}, t-m = {t-m}")
            return 0
        else:
            return 1
    else:
        return 1
    
def T_t_overcount_array_for_given_n(n,t):
    E_t = E_n_over_t_list(n,t)
    T_t_list = list()
    r = int(floor((t-1)/2))+1
    if t == 1:
        T_t_list.append(len(E_t))
    elif t == n:
        for l in range(0,r):
            ratio_factorials = factorial(t-1)/(factorial(t-1-2*l)*factorial(l)*(2**l))
#             try:
#                 test = factorial(t-1)/(factorial(t-1-2*l)*factorial(l)*(2**l))
#                 if int(ratio_factorials)!=int(test):
#                     print()
#                     print(f'for n={n} and t={t}, {ratio_factorials}!={test}')
#                     print()
#             except Exception as e:
#                 print(e)
            T_t_list.append(int(ratio_factorials))
    else:
        for j in E_t:
            K_tj = len(K_tj_list(n,t,j))
            for l in range(0,r):
                Ratio_n_over_t = int(n/t)
                Ratio_n_over_t_to_l = Ratio_n_over_t**l
                ratio_factorials = factorial(t-1)//(factorial(t-1-2*l)*factorial(l)*(2**l))
#                 try:
#                     test = factorial(t-1)/(factorial(t-1-2*l)*factorial(l)*(2**l))
#                     if int(ratio_factorials)!=int(test):
#                         print()
#                         print(f'for n={n} and t={t}, {ratio_factorials}!={test}')
#                         print()
#                 except Exception as e:
#                     print(e)
                K_value = K_tj**(t-1-2*l)
                product = int(Ratio_n_over_t_to_l*ratio_factorials*K_value)
                T_t_list.append(product)
    return T_t_list

def T_t_overcount_for_given_n(n,t):
    T_t_list = T_t_overcount_array_for_given_n(n,t)
    return sum(T_t_list)
    
def T_t_order_2_elements_given_n_and_t(n,t):
    if n%t != 0:
        print(f'{t} does not divide {n}')
    else:
        T1 = len(E_n_over_t_list(n,1))
        divisors_list = factors_of_n(t)
        prime_dictionary = dict([('T1',T1)])
        for divisor in divisors_list:
            divisor_list_of_given_divisor = factors_of_n(divisor)
            s = len(divisor_list_of_given_divisor)
            if s==2:
                p = divisor_list_of_given_divisor[s-1]
                T = T_t_overcount_for_given_n(n,p)
                Tp = T-T1
                prime_dictionary[f'T{p}'] = Tp
            if s>2:
                Tdivisor = T_t_overcount_for_given_n(n,divisor)
                factors_of_divisor = factors_of_n(divisor)
                factors_of_divisor.pop()
                for l in factors_of_divisor:
                    Tdivisor -= prime_dictionary[f'T{l}']
                    prime_dictionary[f'T{divisor}'] = Tdivisor
    return prime_dictionary[f'T{t}']
    

def M_t_overcount_array_for_given_n(n,t):
    E_t = E_n_over_t_list(n,t)
    if t == 1:
        return totient(n)
    if t == n:
        return factorial(n-1)
    else:
        nt = int(n/t)
        Euler = totient(nt)
        power = int(nt**(t-1))
        fact = factorial(t-1)
        return Euler*power*fact
    
    
def M_t_total_elements_given_n_and_t(n,t):
    if n%t != 0:
        print(f'{t} does not divide {n}')
    else:
        M1 = M_t_overcount_array_for_given_n(n,1)
        divisors_list = factors_of_n(t)
        prime_dictionary = dict([('M1',M1)])
        for divisor in divisors_list:
            divisor_list_of_given_divisor = factors_of_n(divisor)
            s = len(divisor_list_of_given_divisor)
            if s==2:
                p = divisor_list_of_given_divisor[s-1]
                M = M_t_overcount_array_for_given_n(n,p)
                Mp = M-M1
                prime_dictionary[f'M{p}'] = Mp
            if s>2:
                Mdivisor = M_t_overcount_array_for_given_n(n,divisor)
                factors_of_divisor = factors_of_n(divisor)
                factors_of_divisor.pop()
                for l in factors_of_divisor:
                    Mdivisor -= prime_dictionary[f'M{l}']
                    prime_dictionary[f'M{divisor}'] = Mdivisor
    return prime_dictionary[f'M{t}']

def involutions_with_stab_gen_by_a_to_t_with_r_fixed_points_overcount(n,t,r):
    if (n-r)%2==1:
        return 0
    else:
        E_set = E_n_over_t_list(n,t)
        if t==1:
            total_is = 0
            for j in E_set:
                beta = gcd(j-1,n)
                if beta == r:
                    total_is +=1
            return total_is
        elif t==n:
            val = int((n-r)/2)
            facto = factorial(n-1)/(factorial(r-1)*(2**val)*factorial(val))
#             try:
#                 test = factorial(n-1)/(factorial(r-1)*(2**val)*factorial(val))
#                 if int(facto)!=int(test):
#                     print()
#                     print(f'for n={n} and t={t}, {facto}!={test}')
#                     print()
#             except Exception as e:
#                 print(e)
            return int(facto)
        else:
            nt = int(n/t)
            values_to_sum = list()
            for j in E_set:
                beta = gcd(j-1,nt)
                if r % beta == 0:
                    if r<= (t*beta):
                        #print(f"(n,t,r) = {(n,t,r)}, beta = {beta}")
                        K_P_intersection_set = K_P_compliment_intersection_set(n,t,j)
                        m = int(r/beta)
                        up_to = floor((t-m)/2)
                        P_set = P_list(n,t,j)
                        for l in range(0,up_to+1):
                            #print(f"j = {j}, complicated delta = {delta_zero_for_P_compliment_set(n,t,j,r,l)}")
                            if delta_zero_for_P_compliment_set(n,t,j,r,l)==1:
                                #print(f"r={r}")
                                P1 = new_power(len(P_set),m-1)
                                P2 = new_power(len(K_P_intersection_set),t-2*l-m)
                                #print(f"P_set = {P_set}, P_set_comp = {K_P_intersection_set}, m-1 = {m-1}, t-2l-m = {t-2*l-m}")
                                binom = factorial(t-1-2*l)/(factorial(m-1)*factorial(t-m-2*l))
                                mult = nt**l
                                fact = factorial(t-1)/(factorial(t-1-2*l)*(2**l)*factorial(l))
                                #print(f"(n,t,r) = {(n,t,r)}, fact = {fact}, P1 = {P1}, P2 = {P2}")
                                value = int(binom)*int(P1)*int(P2)*int(mult)*int(fact)
                                values_to_sum.append(value)
                                #print(f"j = {j}, l = {l}, overcount with l = {value}")
            total = sum(values_to_sum)
            #print(f"(n,t,r) = {(n,t,r)}, over_count = {total}")
            return total
                    
def involutions_with_stab_gen_by_a_to_t_with_r_fixed_points_library_remove_overcount(n,t,r):
    R1 = involutions_with_stab_gen_by_a_to_t_with_r_fixed_points_overcount(n,1,r)
    divisors_list = factors_of_n(t)
    prime_dictionary = dict([('R1',R1)])
    for divisor in divisors_list:
        divisor_list_of_given_divisor = factors_of_n(divisor)
        primeness = len(divisor_list_of_given_divisor)
        if primeness==2:
                p = divisor_list_of_given_divisor[primeness-1]
                R = involutions_with_stab_gen_by_a_to_t_with_r_fixed_points_overcount(n,p,r)
                Rp = R-R1
                prime_dictionary[f'R{p}'] = Rp
        if primeness>2:
                Rdivisor = involutions_with_stab_gen_by_a_to_t_with_r_fixed_points_overcount(n,divisor,r)
                factors_of_divisor = factors_of_n(divisor)
                factors_of_divisor.pop()
                for l in factors_of_divisor:
                    Rdivisor -= prime_dictionary[f'R{l}']
                    prime_dictionary[f'R{divisor}'] = Rdivisor
    return prime_dictionary[f'R{t}']
            
def Permutations_x_with_stab_gen_by_a_to_t_with_r_fixed_points_smaller_than_t_overcount(n,t,r):
    values_to_sum =list()
    nt = int(n/t)
    if (t-r)%2==1:
        return 0
    if n==t:
        fact1 = factorial(n-1)
        fact2 = factorial(r-1)
        fact3 = factorial(int((t-r)/2))
        power = int(2**((t-r)/2))
        fact = fact1/(fact2*fact3*power)
        return int(fact)
    else:   
        ratio = int((t-r)/2)
        E_x = E_n_over_t_list(n,t)
        nt = int(n/t)
        fact1 = factorial(t-1)
        fact2 = factorial(r-1)
        fact3 = factorial(ratio)
        power1 = 2**ratio
        power2 = nt**(ratio)
        power2_int = int(power2)
        value = fact1/(fact2*fact3*power1)
        value_int = int(value)
        for j in E_x:
            alpha = alpha_value(n,t,j)
            power3 = alpha**(r-1)
            power3_int = int(power3)
            final_value = power3_int*power2_int*value_int
            values_to_sum.append(final_value)
        final_sum = sum(values_to_sum)
        return final_sum

def Permutations_x_with_stab_gen_by_a_to_s_dividing_t_with_r_fixed_points_smaller_than_t_constructing_overcount(n,t,s,r):
    if s==1:
        values = 0
        E_sigma_set = E_n_over_t_list(t,1)
        for j_sigma in E_sigma_set:
            Special_E_sigma = Special_E_from_sigma(n,t,1,j_sigma)
            mult = len(Special_E_sigma)
            beta = beta_value(t,1,j_sigma)
            if beta == r:
                values+=mult
        return values
    if t==n:
        return involutions_with_stab_gen_by_a_to_t_with_r_fixed_points_overcount(n,s,r)
    else:
        values_to_remove = list()
        E_sigma = E_n_over_t_list(t,s)
        ns = int(n/s)
        ts = int(t/s)
        for j_sigma in E_sigma:
            Special_E_sigma = Special_E_from_sigma(n,t,s,j_sigma)
            if delta_zero_if(t,s,r,j_sigma)==1:
                for j_prime in Special_E_sigma:
                    P_set = P_list(t,s,j_sigma)
                    PK_set_compliment = K_P_compliment_intersection_set(t,s,j_sigma)
                    beta = beta_value(t,s,j_sigma)
                    m = r/beta
                    m_int = int(m)
                    Special_K_set = Special_K(n,t,s,j_sigma,j_prime)
                    Special_K_compliment_set = Special_K_compliment(n,t,s,j_sigma,j_prime)
                    value = floor((s-m_int)/2)
                    for l in range(0,value+1):
                        P_set_power = new_power(len(P_set),m_int-1)
                        PK_set_power_compliment = new_power(len(PK_set_compliment),s-2*l-m_int)
                        K_set_power = new_power(len(Special_K_set),m_int-1)
                        K_set_power_compliment = new_power(len(Special_K_compliment_set),s-2*l-m_int)
                        facto = int(factorial(s-1-2*l)/(factorial(s-2*l-m_int)*factorial(m_int-1)))
                        value1 = int(ts**l)
                        value2 = int(ns**l)
                        fact = int(factorial(s-1)/(factorial(s-1-2*l)*(2**l)*factorial(l)))
                        total_value = facto*K_set_power*K_set_power_compliment*value2*fact
#                         print(
#                             f"s = {s}, l = {l}" 
#                             f"\n Values in product (P,Pc,K,Kc,facto,fact,ns) = "
#                             f"{P_set_power}, " 
#                             f"{PK_set_power_compliment}, " 
#                             f"{K_set_power}, " 
#                             f"{K_set_power_compliment}, " 
#                             f"{factorial(s-1-2*l)/(factorial(s-2*l-m_int)*factorial(m_int-1))}, "
#                             f"{factorial(s-1)/(factorial(s-1-2*l)*(2**l)*factorial(l))}, "
#                             f"{value2} = {total_value}")
                        values_to_remove.append(total_value)
        value_to_remove = sum(values_to_remove)
        return value_to_remove

def number_of_x_with_stab_gen_by_a_to_t_with_r_fixed_pts_smaller_than_t_library_for_removing_s(n,t,s,r):
    if (t-r)%2==1:
        return 0
    else:
        O1 = Permutations_x_with_stab_gen_by_a_to_s_dividing_t_with_r_fixed_points_smaller_than_t_constructing_overcount(n,t,1,r)
        divisors_list = factors_of_n(s)
        prime_dictionary = dict([('O1',O1)])
        for divisor in divisors_list:
            divisor_list_of_given_divisor = factors_of_n(divisor)
            primeness = len(divisor_list_of_given_divisor)
            if primeness==2:
                    p = divisor_list_of_given_divisor[1]
                    O = Permutations_x_with_stab_gen_by_a_to_s_dividing_t_with_r_fixed_points_smaller_than_t_constructing_overcount(n,t,p,r)
                    Op = O-O1
                    prime_dictionary[f'O{p}'] = Op
            if primeness>2:
                    Odivisor = Permutations_x_with_stab_gen_by_a_to_s_dividing_t_with_r_fixed_points_smaller_than_t_constructing_overcount(n,t,divisor,r)
                    factors_of_divisor = factors_of_n(divisor)
                    factors_of_divisor.pop()
                    for l in factors_of_divisor:
                        Odivisor -= prime_dictionary[f'O{l}']
                        prime_dictionary[f'O{divisor}'] = Odivisor
        return prime_dictionary[f'O{s}']
    
def Permutations_x_with_stab_gen_by_a_to_t_with_r_fixed_points_smaller_than_t_remove_overcount(n,t,r):
    if (t-r)%2==1:
        return 0
    else:
        divisors_list = factors_of_n(t)
        divisors_list.pop()
        overcount = Permutations_x_with_stab_gen_by_a_to_t_with_r_fixed_points_smaller_than_t_overcount(n,t,r)
        #print(f"overcount = {overcount}, divisors of {t} = {divisors_list}")
        list_to_remove = list()
        for s in divisors_list:
            extra = number_of_x_with_stab_gen_by_a_to_t_with_r_fixed_pts_smaller_than_t_library_for_removing_s(n,t,s,r)
            #print(f"s = {s}, extra = {extra}")
            list_to_remove.append(extra)
        summation = sum(list_to_remove)
        return overcount-summation

def orbits_with_r_involutions_with_t_elements(n,t,r):
    if r!=0:
        permutations = Permutations_x_with_stab_gen_by_a_to_t_with_r_fixed_points_smaller_than_t_remove_overcount(n,t,r)
        return int(permutations/r)
    else:
        total_perms = M_t_total_elements_given_n_and_t(n,t)
        total_orbits = int(total_perms/t)
        for r in range(1,t+1):
            perms = Permutations_x_with_stab_gen_by_a_to_t_with_r_fixed_points_smaller_than_t_remove_overcount(n,t,r)
            total_orbits -= int(perms/r)
        return total_orbits

def Permutations_x_with_stab_gen_by_a_to_s_dividing_t_with_r_fixed_points_smaller_than_t_constructing_overcount_with_j(n,t,s,r,j):
    values_to_remove = list()
    E_sigma = E_n_over_t_list(t,s)
    ns = int(n/s)
    ts = int(t/s)
    for j_sigma in E_sigma:
        Special_E_sigma = Special_E_from_sigma(n,t,s,j_sigma)
        if delta_zero_if(t,s,r,j_sigma)==1:
            for j_prime in Special_E_sigma:
                if delta_zero_if_j(n,t,j_prime,j)==1:
                    P_set = P_list(t,s,j_sigma)
                    P_set_compliment = P_list_compliment(t,s,j_sigma)
                    beta = beta_value(t,s,j_sigma)
                    m = r/beta
                    m_int = int(m)
                    Special_K_set = Special_K(n,t,s,j_sigma,j_prime)
                    Special_K_compliment_set = Special_K_compliment(n,t,s,j_sigma,j_prime)
                    value = floor((s-m_int)/2)
                    for l in range(0,value+1):
                        P_set_power = new_power(len(P_set),m_int-1)
                        P_set_power_compliment = new_power(len(P_set_compliment),s-2*l-m_int)
                        K_set_power = new_power(len(Special_K_set),m_int-1)
                        K_set_power_compliment = new_power(len(Special_K_compliment_set),s-2*l-m_int)
                        value1 = int(ts**l)
                        value2 = int(ns**l)
                        fact = int(factorial(s-1)/(factorial(s-1-2*l)*(2**l)*factorial(l)))
                        total_value = P_set_power*P_set_power_compliment*K_set_power*K_set_power_compliment*value2*fact
                        values_to_remove.append(total_value)
    value_to_remove = sum(values_to_remove)
    return value_to_remove

def number_of_x_with_stab_gen_by_a_to_t_with_r_fixed_pts_smaller_than_t_library_for_removing_s_with_j(n,t,s,r,j):
    if (t-r)%2==1:
        return 0
    else:
        O1 = Permutations_x_with_stab_gen_by_a_to_s_dividing_t_with_r_fixed_points_smaller_than_t_constructing_overcount_with_j(n,t,1,r,j)
        divisors_list = factors_of_n(s)
        prime_dictionary = dict([('O1',O1)])
        for divisor in divisors_list:
            divisor_list_of_given_divisor = factors_of_n(divisor)
            primeness = len(divisor_list_of_given_divisor)
            if primeness==2:
                    p = divisor_list_of_given_divisor[primeness-1]
                    O = Permutations_x_with_stab_gen_by_a_to_s_dividing_t_with_r_fixed_points_smaller_than_t_constructing_overcount_with_j(n,t,p,r,j)
                    Op = O-O1
                    prime_dictionary[f'O{p}'] = Op
            if primeness>2:
                    Odivisor = Permutations_x_with_stab_gen_by_a_to_s_dividing_t_with_r_fixed_points_smaller_than_t_constructing_overcount_with_j(n,t,divisor,r,j)
                    factors_of_divisor = factors_of_n(divisor)
                    factors_of_divisor.pop()
                    for l in factors_of_divisor:
                        Odivisor -= prime_dictionary[f'O{l}']
                        prime_dictionary[f'O{divisor}'] = Odivisor
        return prime_dictionary[f'O{s}']

def orbits_with_t_elements_and_r_involutions_where_indicator_is_plus_one_with_j(n,t,r,j):
    #print(f"r = {r}")
    if (t-r)%2==1:
        return 0
    if n==t:
        return orbits_with_r_involutions_with_t_elements(n,t,r)
    else:
        alpha = alpha_value(n,t,j)
        nt = int(n/t)
        power = int((t-r)/2)
        fact = factorial(t-1)/(factorial(r-1)*(2**power)*factorial(power))
        overcount = int((alpha**(r-1))*(nt**power)*fact)
        values_to_remove = list()
        divisors_list = factors_of_n(t)
        divisors_list.pop()
        for s in divisors_list:
            value = number_of_x_with_stab_gen_by_a_to_t_with_r_fixed_pts_smaller_than_t_library_for_removing_s_with_j(n,t,s,r,j)
            values_to_remove.append(value)
    remove_overcount = sum(values_to_remove)
    total = overcount - remove_overcount
    total_scaled = int(total/r)
    return total_scaled

def number_representations_with_indicator_plus_one(n,t):
    E_set = E_n_over_t_list(n,t)
    sum_list = list()
    if len(E_set) == 0:
        for r in range(1,n+1):
            value = orbits_with_r_involutions_with_t_elements(n,n,r)
            sum_list.append(value*n)
    else:
        for j in E_set:
            alpha = alpha_value(n,t,j)
            for r in range(1,t+1):
                value = alpha*orbits_with_t_elements_and_r_involutions_where_indicator_is_plus_one_with_j(n,t,r,j)
                sum_list.append(t*value)
    total = sum(sum_list)
    return total

def K_set_t_2(n,j):
    n2 = int(n/2)
    K_set = list()
    for k in range(0,n2):
        value = k*(j+1)
        value_mod = value%n2
        if value_mod == 0:
            K_set.append(k)
    return K_set

def list_indicators_for_permutations_when_t_is_2_with_inverse_inside(n):
    n2 = int(n/2)
    plus_ones_total = 0
    minus_ones_total = 0
    zeros_total = 0
    E_set = E_n_over_t_list(n,2)
    for j in E_set:
        for k in range(0,n2):
            k_value = (2*k+1)%n2
            if k_value !=j:
                K_set = K_set_t_2(n,j)
                k_check = (k+1)*(j-1)
                k_check_mod = k_check % n2
                if k in K_set:
                    for i in range(0,n2):
                        i_value = (i*(j+1))%n2
                        if i_value == 0:
                            plus_ones_total+=1
                        else:
                            zeros_total+=1
                if k_check_mod==0:
                    for i in range(0,n2):
                        i_value2 = (i*(j+1))%n2
                        if i_value2 == 0:
                            ik_value = (-i*(k+1))%n2
                            if ik_value == 0:
                                plus_ones_total+=1
                            else:
                                minus_ones_total+=1
                        else:
                            zeros_total+=1
    return [plus_ones_total,minus_ones_total,zeros_total]

def total_permutations_when_t_is_2_with_inverse_without_inverse(n):
    Mt = M_t_total_elements_given_n_and_t(n,2)
    Ind_with_inverse_list = list_indicators_for_permutations_when_t_is_2_with_inverse_inside(n)
    Ind_with_inverse_total = sum(Ind_with_inverse_list)
    perms_with_inverse = int(Ind_with_inverse_total*2/n)
    perms_without_inverse = Mt-perms_with_inverse
    return print(f"When t = 2, there are {perms_with_inverse} permutations with their inverse in their orbit, and {perms_without_inverse} without their inverse in their orbit.")


def Final_Indicator_list_for_permutations_when_t_is_2(n):
    Total_ind = int(M_t_total_elements_given_n_and_t(n,2)*n/2)
    Ind_with_inverse_list = list_indicators_for_permutations_when_t_is_2_with_inverse_inside(n)
    Ind_with_inverse_total = sum(Ind_with_inverse_list)
    Total_zeros_from_ind_without_inverse = Total_ind-Ind_with_inverse_total
    return [Ind_with_inverse_list[0],Ind_with_inverse_list[1],Ind_with_inverse_list[2]+Total_zeros_from_ind_without_inverse ]

## Ratio Calculations

In [ ]:
def ratio_plus_one_indicator(n,t):
    total_irreps = int(n/t)*M_t_total_elements_given_n_and_t(n,t)
    if t%2==1:
        irreps_with_plus_one_ind = number_representations_with_indicator_plus_one(n,t)
        ratio1 = irreps_with_plus_one_ind/total_irreps
        return ratio1*100
    if t==2:
        irreps_with_plus_one_ind_t2 = Final_Indicator_list_for_permutations_when_t_is_2(n)[0]
        ratio2 = irreps_with_plus_one_ind_t2/total_irreps
        return ratio2*100
    else:
        irreps_with_plus_one_ind_direct = total_indicators_for_irreps_dim_t_given_n_as_list_plus_minus_zero(n,t)[0]
        ratio3 = irreps_with_plus_one_ind_direct/total_irreps
        return ratio3*100
    
def ratio_minus_one_indicator(n,t):
    total_irreps = int(n/t)*M_t_total_elements_given_n_and_t(n,t)
    if t==2:
        irreps_with_plus_one_ind_t2 = Final_Indicator_list_for_permutations_when_t_is_2(n)[1]
        ratio2 = irreps_with_plus_one_ind_t2/total_irreps
        return ratio2*100
    else:
        irreps_with_plus_one_ind_direct = total_indicators_for_irreps_dim_t_given_n_as_list_plus_minus_zero(n,t)[1]
        ratio3 = irreps_with_plus_one_ind_direct/total_irreps
        return ratio3*100

# Generating Data Frames and Graphs

### Permutations with Stabilizer size n/t

In [ ]:
Total_Perms_data = []
for n in range(1,50):
    for t in factors_of_n(n):
        print(f'working on ({n},{t})...')
        if t<8:
            print('calculating...')
            Total_Perms_data.append([
                'My Formula',
                f'({n},{t})',
                M_t_total_elements_given_n_and_t(n,t)])
            Total_Perms_data.append([
                'Direct Computation',
                f'({n},{t})',
                len(get_unique_orbits(n,t))*t])
        else:
            print(f'{t} is too big, yuck!')

In [ ]:
total_perms_df = pd.DataFrame(Total_Perms_data, columns = ['func', 'input', 'result'])
total_perms_df = total_perms_df[~total_perms_df['input'].isin(list(total_perms_df[total_perms_df['result']==0]['input']))]
total_perms_df

In [ ]:
total_perm_direct_only_df = total_perms_df[(total_perms_df['func'] != 'My Formula')]
total_perm_direct_only_df

In [ ]:
fig1 = px.line(total_perm_direct_only_df, x="input", y="result", log_y=True, labels={
                     "input": "(n,t)",
                     "result": "Total Number of Permutations with Stabilizer Size n/t"
                      },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of permutations in symmetric group of order n-1 with stabilizer size n/t</sup>"
                      )
fig1.update_traces(patch={"line": {"color": "darkred", "width": 2}})

ignore_if = 0
for i in range(1,51):
    if list(total_perm_direct_only_df['result'])[i+1]*100<list(total_perm_direct_only_df['result'])[i]>100*list(total_perm_direct_only_df['result'])[i-1]:
        ignore_if+=1
        #if ignore_if%2==1:
        if i!=41:
            fig1.add_annotation(
                    x=list(total_perm_direct_only_df['input'])[i],
                    y=math.log10(list(total_perm_direct_only_df['result'])[i]),
                    xref="x",
                    yref="y",
                    text=f"(n,t)={list(total_perm_direct_only_df['input'])[i]},Total={int(list(total_perm_direct_only_df['result'])[i])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=15,
                    ay=-50-i/5,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#854f42",
                    opacity=0.6,
                    textangle=12
                    )
            
for i in range(51,115):
    if list(total_perm_direct_only_df['result'])[i+1]*100<list(total_perm_direct_only_df['result'])[i]>100*list(total_perm_direct_only_df['result'])[i-1]:
        ignore_if+=1
        if ignore_if%3!=1:
            if i not in [59,66,78,82,104]:
                fig1.add_annotation(
                        x=list(total_perm_direct_only_df['input'])[i],
                        y=math.log10(list(total_perm_direct_only_df['result'])[i]),
                        xref="x",
                        yref="y",
                        text=f"(n,t)={list(total_perm_direct_only_df['input'])[i]},Total={int(list(total_perm_direct_only_df['result'])[i])}",
                        showarrow=True,
                        font=dict(
                            family="Courier New, monospace",
                            size=10,
                            color="#ffffff"
                            ),
                        align="center",
                        arrowhead=2,
                        arrowsize=1,
                        arrowwidth=2,
                        arrowcolor="#636363",
                        ax=15,
                        ay=-50-i/5,
                        bordercolor="#c7c7c7",
                        borderwidth=2,
                        borderpad=4,
                        bgcolor="#854f42",
                        opacity=0.6,
                        textangle=12
                        )

fig1.add_annotation(
                    x=list(total_perm_direct_only_df['input'])[59],
                    y=math.log10(list(total_perm_direct_only_df['result'])[59]),
                    xref="x",
                    yref="y",
                    text=f"(n,t)={list(total_perm_direct_only_df['input'])[59]}, Total={int(list(total_perm_direct_only_df['result'])[59])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=15,
                    ay=-75,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#854f42",
                    opacity=0.6,
                    textangle=12
                    )


fig1.add_annotation(
                    x=list(total_perm_direct_only_df['input'])[82],
                    y=math.log10(list(total_perm_direct_only_df['result'])[82]),
                    xref="x",
                    yref="y",
                    text=f"(n,t)={list(total_perm_direct_only_df['input'])[82]}, Total={int(list(total_perm_direct_only_df['result'])[82])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=15,
                    ay=-85,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#854f42",
                    opacity=0.6,
                    textangle=12
                    )

fig1.add_annotation(
                    x=list(total_perm_direct_only_df['input'])[41],
                    y=math.log10(list(total_perm_direct_only_df['result'])[41]),
                    xref="x",
                    yref="y",
                    text=f"(n,t)={list(total_perm_direct_only_df['input'])[41]}, Total={int(list(total_perm_direct_only_df['result'])[41])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=-45,
                    ay=-80,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#854f42",
                    opacity=0.6,
                    textangle=12
                    )

fig1.add_annotation(
                    x=list(total_perm_direct_only_df['input'])[87],
                    y=math.log10(list(total_perm_direct_only_df['result'])[87]),
                    xref="x",
                    yref="y",
                    text=f"Maximum: (n,t)={list(total_perm_direct_only_df['input'])[87]}, Total={int(list(total_perm_direct_only_df['result'])[87])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=15,
                    ay=-100,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#701212",
                    opacity=0.6,
                    #textangle=12
                    )
                
fig1.show()


In [ ]:
M_t_total_elements_given_n_and_t(14,7)

In [ ]:
my_list = list(total_perms_df[total_perms_df['func']=='My Formula']['result'])
direct_computation = list(total_perms_df[total_perms_df['func']!='My Formula']['result'])

data_diff1 = {'Total Me':my_list,
        'Total Direct':direct_computation}
 
# Create DataFrame
total_perms_df_difference = pd.DataFrame(data_diff1)

total_perms_df_difference['difference']=total_perms_df_difference['Total Direct']-total_perms_df_difference['Total Me']
difference = list(total_perms_df_difference['difference'])
total_perms_df_difference['input'] = list(total_perms_df['input'].unique())
total_perms_df_difference[~total_perms_df_difference['input'].isin(list(total_perms_df_difference[total_perms_df_difference['difference']==0]['input']))]

In [ ]:
fig2 = px.line(total_perms_df_difference, x="input", y="difference", labels={
                     "input": "(n,t)",
                     "difference": "Difference: Direct Computation - My Formula"},
                    title="Difference Between Direct Computation and My Formula: <br><sup> Difference is zero for all values calculated</sup>")
#fig2.update_traces(patch={"line": {"color": "black", "width": 4}})
#fig2.show()

### Involutions with stabilizer size n/t

In [ ]:
Total_inv_data = []
for n in range(1,50):
    for t in factors_of_n(n):
        print(f'working on ({n},{t})...')
        if t<8:
            print('calculating...')
            Total_inv_data.append([
                'My Formula',
                f'({n},{t})',
                T_t_order_2_elements_given_n_and_t(n,t)])
            Total_inv_data.append([
                'Direct Computation',
                f'({n},{t})',
                len(list_involutions(n,t))])
        else:
            print(f'{t} is too big, yuck!')

In [ ]:
total_inv_df = pd.DataFrame(Total_inv_data, columns = ['func', 'input', 'result'])
total_inv_df = total_inv_df[~total_inv_df['input'].isin(list(total_inv_df[total_inv_df['result']==0]['input']))]
total_inv_df

In [ ]:
total_inv_mine_only_df = total_inv_df[(total_inv_df['func'] == 'My Formula')]
total_inv_mine_only_df

In [ ]:
fig3 = px.line(total_inv_mine_only_df, x="input", y="result", log_y=True, labels={
                     "input": "(n,t)",
                     "result": "Total Number of Involutions with Stabilizer Size n/t"
                     },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of involutions in symmetric group of order n-1 with stabilizer size n/t</sup>"
              )
fig3.update_traces(patch={"line": {"color": "darkgoldenrod", "width": 2}})
# fig.update_traces(patch={"line": {"color": "blue", "width": 4, "dash": 'dot'}}, selector={"legendgroup": "Direct Computation"})

ignore_if = 0
for i in range(1,114):
    if list(total_inv_mine_only_df['result'])[i+1]*10<list(total_inv_mine_only_df['result'])[i]>10*list(total_inv_mine_only_df['result'])[i-1]:
        ignore_if+=1
        if ignore_if%2==1:
            if i not in [40,62,75,81,91]:
                fig3.add_annotation(
                        x=list(total_inv_mine_only_df['input'])[i],
                        y=math.log10(list(total_inv_mine_only_df['result'])[i]),
                        xref="x",
                        yref="y",
                            text=f"(n,t)={list(total_inv_mine_only_df['input'])[i]},Total={int(list(total_inv_mine_only_df['result'])[i])}",
                        showarrow=True,
                        font=dict(
                            family="Courier New, monospace",
                            size=10,
                            color="#ffffff"
                            ),
                        align="center",
                        arrowhead=2,
                        arrowsize=1,
                        arrowwidth=2,
                        arrowcolor="#636363",
                        ax=10,
                        ay=-55-i/5,
                        bordercolor="#c7c7c7",
                        borderwidth=2,
                        borderpad=4,
                        bgcolor="#b38b09",
                        opacity=0.6,
                        textangle=5
                        )

fig3.add_annotation(
                    x=list(total_inv_mine_only_df['input'])[40],
                    y=math.log10(list(total_inv_mine_only_df['result'])[40]),
                    xref="x",
                    yref="y",
                    text=f"(n,t)={list(total_inv_mine_only_df['input'])[40]}, Total={int(list(total_inv_mine_only_df['result'])[40])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=15,
                    ay=-75,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#b38b09",
                    opacity=0.6,
                    textangle=5
                    )

fig3.add_annotation(
                    x=list(total_inv_mine_only_df['input'])[75],
                    y=math.log10(list(total_inv_mine_only_df['result'])[75]),
                    xref="x",
                    yref="y",
                    text=f"(n,t)={list(total_inv_mine_only_df['input'])[75]}, Total={int(list(total_inv_mine_only_df['result'])[75])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=-35,
                    ay=-160,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#b38b09",
                    opacity=0.6,
                    textangle=5
                    )

fig3.add_annotation(
                    x=list(total_inv_mine_only_df['input'])[81],
                    y=math.log10(list(total_inv_mine_only_df['result'])[81]),
                    xref="x",
                    yref="y",
                    text=f"(n,t)={list(total_inv_mine_only_df['input'])[81]}, Total={int(list(total_inv_mine_only_df['result'])[81])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=15,
                    ay=-85,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#b38b09",
                    opacity=0.6,
                    textangle=5
                    )

fig3.add_annotation(
                    x=list(total_inv_mine_only_df['input'])[86],
                    y=math.log10(list(total_inv_mine_only_df['result'])[86]),
                    xref="x",
                    yref="y",
                    text=f"Maximum: (n,t)={list(total_inv_mine_only_df['input'])[86]}, Total={int(list(total_inv_mine_only_df['result'])[86])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=15,
                    ay=-105,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#99440b",
                    opacity=0.6,
                    #textangle=5
                    )
    
fig3.show()


In [ ]:
my_list2 = list(total_inv_df[total_inv_df['func']=='My Formula']['result'])
direct_computation2 = list(total_inv_df[total_inv_df['func']!='My Formula']['result'])

data_diff2 = {'Involutions Me':my_list2,
        'Involutions Direct':direct_computation2}
 
# Create DataFrame
total_inv_df_difference = pd.DataFrame(data_diff2)

total_inv_df_difference['difference']=total_inv_df_difference['Involutions Direct']-total_inv_df_difference['Involutions Me']
difference = list(total_inv_df_difference['difference'])
total_inv_df_difference['input'] = list(total_inv_df['input'].unique())
total_inv_df_difference[~total_inv_df_difference['input'].isin(list(total_inv_df_difference[total_inv_df_difference['difference']==0]['input']))]

In [ ]:
fig4 = px.line(total_inv_df_difference, x="input", y="difference", labels={
                     "input": "(n,t)",
                     "difference": "Difference: Direct Computation - My Formula"},
                    title="Difference Between Direct Computation and My Formula: <br><sup> Difference is zero for all values calcualted </sup>")
# fig4.update_traces(patch={"line": {"color": "black", "width": 4}})
# fig4.show()

### Involutions with stabilizer size n/t with r fixed points

In [ ]:
Total_inv_data_with_fixed = []
for n in range(1,20):
    for t in factors_of_n(n):
        for r in range(0,n+1):
            print(f'working on ({n},{t},{r})...')
            if t<10:
                print('calculating...')
                Total_inv_data_with_fixed.append([
                    'My Formula',
                    f'({n},{t},{r})',
                    involutions_with_stab_gen_by_a_to_t_with_r_fixed_points_library_remove_overcount(n,t,r)])
                Total_inv_data_with_fixed.append([
                    'Direct Computation',
                    f'({n},{t},{r})',
                    len(list_involutions_with_r_fixed_points(n,t,r))])
            else:
                print(f'{t} is too big, yuck!')

In [ ]:
total_inv_fixed_df = pd.DataFrame(Total_inv_data_with_fixed, columns = ['func', 'input', 'result'])
total_inv_fixed_df = total_inv_fixed_df[~total_inv_fixed_df['input'].isin(list(total_inv_fixed_df[total_inv_fixed_df['result']==0]['input']))]
total_inv_fixed_df

In [ ]:
total_inv_fixed_mine_only_df = total_inv_fixed_df[(total_inv_fixed_df['func'] == 'My Formula')]
total_inv_fixed_mine_only_df

In [ ]:
fig5 = px.line(total_inv_fixed_mine_only_df, x="input", y="result", log_y=True, labels={
                     "input": "(n,t,r)",
                     "result": "# Involutions with r Fixed Points and Stabilizer Size n/t",
                     },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of involutions with r fixed points in symmetric group of order n-1 with stabilizer size n/t</sup>"
              )
fig5.update_traces(patch={"line": {"color": "olivedrab", "width": 2}})
# fig.update_traces(patch={"line": {"color": "blue", "width": 4, "dash": 'dot'}}, selector={"legendgroup": "Direct Computation"})

ignore_if = 0
for i in range(1,114):
    if list(total_inv_fixed_mine_only_df['result'])[i+1]*1<list(total_inv_fixed_mine_only_df['result'])[i]>1*list(total_inv_fixed_mine_only_df['result'])[i-1]:
        ignore_if+=1
        if ignore_if%2==1:
            if i not in [5,15,27,56,96,110]:
                fig5.add_annotation(
                        x=list(total_inv_fixed_mine_only_df['input'])[i],
                        y=math.log10(list(total_inv_fixed_mine_only_df['result'])[i]),
                        xref="x",
                        yref="y",
                            text=f"(n,t,r)={list(total_inv_fixed_mine_only_df['input'])[i]},Total={int(list(total_inv_fixed_mine_only_df['result'])[i])}",
                        showarrow=True,
                        font=dict(
                            family="Courier New, monospace",
                            size=10,
                            color="#ffffff"
                            ),
                        align="center",
                        arrowhead=2,
                        arrowsize=1,
                        arrowwidth=2,
                        arrowcolor="#636363",
                        ax=10,
                        ay=-55-i/5,
                        bordercolor="#c7c7c7",
                        borderwidth=2,
                        borderpad=4,
                        bgcolor="#3ea832",
                        opacity=0.6,
                        textangle=5
                        )

fig5.add_annotation(
                    x=list(total_inv_fixed_mine_only_df['input'])[5],
                    y=math.log10(list(total_inv_fixed_mine_only_df['result'])[5]),
                    xref="x",
                    yref="y",
                    text=f"(n,t,r)={list(total_inv_fixed_mine_only_df['input'])[5]}, Total={int(list(total_inv_fixed_mine_only_df['result'])[5])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=-15,
                    ay=-145,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#3ea832",
                    opacity=0.6,
                    textangle=5
                    )

fig5.add_annotation(
                    x=list(total_inv_fixed_mine_only_df['input'])[15],
                    y=math.log10(list(total_inv_fixed_mine_only_df['result'])[15]),
                    xref="x",
                    yref="y",
                    text=f"(n,t,r)={list(total_inv_fixed_mine_only_df['input'])[15]}, Total={int(list(total_inv_fixed_mine_only_df['result'])[15])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=-15,
                    ay=-125,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#3ea832",
                    opacity=0.6,
                    textangle=5
                    )

fig5.add_annotation(
                    x=list(total_inv_fixed_mine_only_df['input'])[29],
                    y=math.log10(list(total_inv_fixed_mine_only_df['result'])[29]),
                    xref="x",
                    yref="y",
                    text=f"(n,t,r)={list(total_inv_fixed_mine_only_df['input'])[29]}, Total={int(list(total_inv_fixed_mine_only_df['result'])[29])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=15,
                    ay=-85,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#3ea832",
                    opacity=0.6,
                    textangle=5
                    )


fig5.add_annotation(
                    x=list(total_inv_fixed_mine_only_df['input'])[63],
                    y=math.log10(list(total_inv_fixed_mine_only_df['result'])[63]),
                    xref="x",
                    yref="y",
                    text=f"(n,t,r)={list(total_inv_fixed_mine_only_df['input'])[63]}, Total={int(list(total_inv_fixed_mine_only_df['result'])[63])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=-35,
                    ay=-25,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#3ea832",
                    opacity=0.6,
                    textangle=-50
                    )

fig5.add_annotation(
                    x=list(total_inv_fixed_mine_only_df['input'])[96],
                    y=math.log10(list(total_inv_fixed_mine_only_df['result'])[96]),
                    xref="x",
                    yref="y",
                    text=f"(n,t,r)={list(total_inv_fixed_mine_only_df['input'])[96]}, Total={int(list(total_inv_fixed_mine_only_df['result'])[96])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=-35,
                    ay=-90,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#3ea832",
                    opacity=0.6,
                    textangle=5
                    )


fig5.add_annotation(
                    x=list(total_inv_fixed_mine_only_df['input'])[110],
                    y=math.log10(list(total_inv_fixed_mine_only_df['result'])[110]),
                    xref="x",
                    yref="y",
                    text=f"Maximum: (n,t,r)={list(total_inv_fixed_mine_only_df['input'])[110]}, Total={int(list(total_inv_fixed_mine_only_df['result'])[110])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=-35,
                    ay=-70,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#155e0c",
                    opacity=0.6,
                    #textangle=5
                    )


fig5.show()



In [ ]:
my_list3 = list(total_inv_fixed_df[total_inv_fixed_df['func']=='My Formula']['result'])
direct_computation3 = list(total_inv_fixed_df[total_inv_fixed_df['func']!='My Formula']['result'])

data_diff3 = {'Involutions with Fixed Points Me':my_list3,
        'Involutions with Fixed Points Direct':direct_computation3}
 
# Create DataFrame
total_inv_fixed_df_difference = pd.DataFrame(data_diff3)

total_inv_fixed_df_difference['difference']=total_inv_fixed_df_difference['Involutions with Fixed Points Direct']-total_inv_fixed_df_difference['Involutions with Fixed Points Me']
difference = list(total_inv_fixed_df_difference['difference'])
total_inv_fixed_df_difference['input'] = list(total_inv_fixed_df['input'].unique())
total_inv_fixed_df_difference[~total_inv_fixed_df_difference['input'].isin(list(total_inv_fixed_df_difference[total_inv_fixed_df_difference['difference']==0]['input']))]

In [ ]:
fig6 = px.line(total_inv_fixed_df_difference, x="input", y="difference", labels={
                     "input": "(n,t,r)",
                     "difference": "Difference: Direct Computation - My Formula"},
                    title="Difference Between Direct Computation and My Formula: <br><sup> Difference is zero for all values calcualted </sup>")
# fig6.update_traces(patch={"line": {"color": "black", "width": 4}})
# fig6.show()

### Orbits of length t containing r involutions

In [ ]:
Total_orbit_data_with_inv = []
for n in range(1,20):
    for t in factors_of_n(n):
        for r in range(0,t+1):
            print(f'working on ({n},{t},{r})...')
            if t<10:
                print('calculating...')
                Total_orbit_data_with_inv.append([
                    'My Formula',
                    f'({n},{t},{r})',
                    orbits_with_r_involutions_with_t_elements(n,t,r)])
                Total_orbit_data_with_inv.append([
                    'Direct Computation',
                    f'({n},{t},{r})',
                    count_orbits_with_i_involutions(n,t,r)])
            else:
                print(f'{t} is too big, yuck!')

In [ ]:
total_orb_with_inv_df = pd.DataFrame(Total_orbit_data_with_inv, columns = ['func', 'input', 'result'])
total_orb_with_inv_df = total_orb_with_inv_df[~total_orb_with_inv_df['input'].isin(list(total_orb_with_inv_df[total_orb_with_inv_df['result']==0]['input']))]
total_orb_with_inv_df

In [ ]:
total_orb_mine_only_df = total_orb_with_inv_df[(total_orb_with_inv_df['func'] == 'My Formula')]
total_orb_mine_only_df

In [ ]:
fig7 = px.line(total_orb_mine_only_df, x="input", y="result", log_y=True, labels={
                     "input": "(n,t,r)",
                     "result": "Total Number of Orbits of length t with r involutions",
                     },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of orbits of length t containing r involutions</sup>"
              )
fig7.update_traces(patch={"line": {"color": "darkgreen", "width": 2}})
# fig.update_traces(patch={"line": {"color": "blue", "width": 4, "dash": 'dot'}}, selector={"legendgroup": "Direct Computation"})

ignore_if = 0
for i in range(1,107):
    if list(total_orb_mine_only_df['result'])[i+1]*4<list(total_orb_mine_only_df['result'])[i]>4*list(total_orb_mine_only_df['result'])[i-1]:
        ignore_if+=1
        #if ignore_if%2==1:
        if i in [38,72]:
            fig7.add_annotation(
                    x=list(total_orb_mine_only_df['input'])[i],
                    y=math.log10(list(total_orb_mine_only_df['result'])[i]),
                    xref="x",
                    yref="y",
                        text=f"(n,t,r)={list(total_orb_mine_only_df['input'])[i]},Total={int(list(total_orb_mine_only_df['result'])[i])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=10,
                    ay=-65,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#14a66e",
                    opacity=0.6,
                    textangle=5
                    )


fig7.add_annotation(
    x=list(total_orb_mine_only_df['input'])[19],
    y=math.log10(list(total_orb_mine_only_df['result'])[19]),
    xref="x",
    yref="y",
    text=f"(n,t,r)={list(total_orb_mine_only_df['input'])[19]}, Total={int(list(total_orb_mine_only_df['result'])[19])}",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="#ffffff"
        ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=-35,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor="#14a66e",
    opacity=0.6,
    textangle=5
    )            
            
fig7.add_annotation(
    x=list(total_orb_mine_only_df['input'])[29],
    y=math.log10(list(total_orb_mine_only_df['result'])[29]),
    xref="x",
    yref="y",
    text=f"(n,t,r)={list(total_orb_mine_only_df['input'])[29]}, Total={int(list(total_orb_mine_only_df['result'])[29])}",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="#ffffff"
        ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=-45,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor="#14a66e",
    opacity=0.6,
    textangle=5
    )

fig7.add_annotation(
    x=list(total_orb_mine_only_df['input'])[47],
    y=math.log10(list(total_orb_mine_only_df['result'])[47]),
    xref="x",
    yref="y",
    text=f"(n,t,r)={list(total_orb_mine_only_df['input'])[47]}, Total={int(list(total_orb_mine_only_df['result'])[47])}",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="#ffffff"
        ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=40,
    ay=-95,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor="#14a66e",
    opacity=0.6,
    textangle=5
    )

fig7.add_annotation(
    x=list(total_orb_mine_only_df['input'])[62],
    y=math.log10(list(total_orb_mine_only_df['result'])[62]),
    xref="x",
    yref="y",
    text=f"(n,t,r)={list(total_orb_mine_only_df['input'])[62]}, Total={int(list(total_orb_mine_only_df['result'])[62])}",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="#ffffff"
        ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=-35,
    ay=-75,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor="#14a66e",
    opacity=0.6,
    textangle=5
    )

fig7.add_annotation(
    x=list(total_orb_mine_only_df['input'])[82],
    y=math.log10(list(total_orb_mine_only_df['result'])[82]),
    xref="x",
    yref="y",
    text=f"(n,t,r)={list(total_orb_mine_only_df['input'])[82]}, Total={int(list(total_orb_mine_only_df['result'])[82])}",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="#ffffff"
        ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=5,
    ay=-95,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor="#14a66e",
    opacity=0.6,
    textangle=5
    )

fig7.add_annotation(
    x=list(total_orb_mine_only_df['input'])[90],
    y=math.log10(list(total_orb_mine_only_df['result'])[90]),
    xref="x",
    yref="y",
    text=f"(n,t,r)={list(total_orb_mine_only_df['input'])[90]}, Total={int(list(total_orb_mine_only_df['result'])[90])}",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="#ffffff"
        ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=-15,
    ay=-55,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor="#14a66e",
    opacity=0.6,
    textangle=35
    )

fig7.add_annotation(
    x=list(total_orb_mine_only_df['input'])[102],
    y=math.log10(list(total_orb_mine_only_df['result'])[102]),
    xref="x",
    yref="y",
    text=f"Maximum: (n,t,r)={list(total_orb_mine_only_df['input'])[102]}, Total={int(list(total_orb_mine_only_df['result'])[102])}",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="#ffffff"
        ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=-55,
    ay=-85,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor="#085437",
    opacity=0.6,
    #textangle=5
    )

fig7.show()


In [ ]:
my_list4 = list(total_orb_with_inv_df[total_orb_with_inv_df['func']=='My Formula']['result'])
direct_computation4 = list(total_orb_with_inv_df[total_orb_with_inv_df['func']!='My Formula']['result'])

data_diff4 = {'Orbits Me':my_list4,
        'Orbits Direct':direct_computation4}
 
# Create DataFrame
total_orb_with_inv_df_difference = pd.DataFrame(data_diff4)

total_orb_with_inv_df_difference['difference']=total_orb_with_inv_df_difference['Orbits Me']-total_orb_with_inv_df_difference['Orbits Direct']
difference = list(total_orb_with_inv_df_difference['difference'])
total_orb_with_inv_df_difference['input'] = list(total_orb_with_inv_df['input'].unique())
total_orb_with_inv_df_difference[~total_orb_with_inv_df_difference['input'].isin(list(total_orb_with_inv_df_difference[total_orb_with_inv_df_difference['difference']==0]['input']))]

In [ ]:
fig8 = px.line(total_inv_fixed_df_difference, x="input", y="difference", labels={
                     "input": "(n,t,r)",
                     "difference": "Difference: Direct Computation - My Formula"},
                    title="Difference Between Direct Computation and My Formula: <br><sup> Difference is zero for all values calcualted </sup>")
# fig8.update_traces(patch={"line": {"color": "black", "width": 4}})
# fig8.show()

### Number of irreps of Jn with indicator +1

In [ ]:
Total_plus_one_irreps_data = []
for n in range(1,50):
    for t in factors_of_n(n):
        print(f'working on ({n},{t})...')
        if t<9:
            print('calculating...')
            Total_plus_one_irreps_data.append([
                'My Formula',
                f'({n},{t})',
                number_representations_with_indicator_plus_one(n,t)])
            Total_plus_one_irreps_data.append([
                'Direct Computation',
                f'({n},{t})',
                total_indicators_for_irreps_dim_t_given_n_as_list_plus_minus_zero(n,t)[0]])
        else:
            print(f'{t} is too big, yuck!')

In [ ]:
total_plus_one_irrep_df = pd.DataFrame(Total_plus_one_irreps_data, columns = ['func', 'input', 'result'])
total_plus_one_irrep_df = total_plus_one_irrep_df[~total_plus_one_irrep_df['input'].isin(list(total_plus_one_irrep_df[total_plus_one_irrep_df['result']==0]['input']))]
total_plus_one_irrep_df

In [ ]:
total_plus_one_irrep_df['func'] = total_plus_one_irrep_df['func'].str.replace('My Formula', 'Our Formula', regex=False)

In [ ]:
fig9 = px.line(total_plus_one_irrep_df, x="input", y="result", log_y=True, color='func', labels={
                     "input": "(n,t)",
                     "result": "Total Number of irreps of Jn of dimension t with indicator +1",
                     "func": "Two Methods of Computation"},
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of irreps of Jn of dimension t with indicator +1</sup>"
                      )
fig9.update_traces(patch={"line": {"color": "darkcyan", "width": 2}}, selector={"legendgroup": "Direct Computation"})
fig9.update_traces(patch={"line": {"color": "lightcoral", "width": 2}}, selector={"legendgroup": "Our Formula"})


fig9.add_annotation(
    x=list(total_plus_one_irrep_df['input'])[176],
    y=math.log10(list(total_plus_one_irrep_df['result'])[176]),
    xref="x",
    yref="y",
    text=f"Maximum: (n,t,r)={list(total_plus_one_irrep_df['input'])[176]}, \n Underestimate={int(list(total_plus_one_irrep_df['result'])[176])}, \n Total={int(list(total_plus_one_irrep_df['result'])[177])}",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="#ffffff"
        ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=-55,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor="#0b6478",
    opacity=0.6,
    #textangle=5
    )           



fig9.show()


In [ ]:
my_list5 = list(total_plus_one_irrep_df[total_plus_one_irrep_df['func']=='My Formula']['result'])
direct_computation5 = list(total_plus_one_irrep_df[total_plus_one_irrep_df['func']!='My Formula']['result'])

data_diff5 = {'Plus Ones Me':my_list5,
        'Plus Ones Direct':direct_computation5}
 
# Create DataFrame
total_plus_one_irrep_df_difference = pd.DataFrame(data_diff5)

total_plus_one_irrep_df_difference['difference']=total_plus_one_irrep_df_difference['Plus Ones Direct']-total_plus_one_irrep_df_difference['Plus Ones Me']
difference = list(total_plus_one_irrep_df_difference['difference'])
total_plus_one_irrep_df_difference['input'] = list(total_plus_one_irrep_df['input'].unique())
total_plus_one_irrep_df_difference = total_plus_one_irrep_df_difference[~total_plus_one_irrep_df_difference['input'].isin(list(total_plus_one_irrep_df_difference[total_plus_one_irrep_df_difference['difference']==0]['input']))]
total_plus_one_irrep_df_difference

In [ ]:
fig_dif = px.line(total_plus_one_irrep_df_difference, x="input", y="difference", log_y=True, labels={
                     "input": "(n,t)",
                     "difference": "Difference: Direct Computation - Our Formula"},
                    #title="Difference Between Direct Computation and My Formula: <br><sup> My count for irreps with indicator +1 is an underestimate for t even </sup>"
                 )
fig_dif.update_traces(patch={"line": {"color": "black", "width": 4}})

for i in range(1,38):
    if i not in [18,23,25]:
        if list(total_plus_one_irrep_df_difference['difference'])[i+1]*10<list(total_plus_one_irrep_df_difference['difference'])[i]>10*list(total_plus_one_irrep_df_difference['difference'])[i-1]:
            fig_dif.add_annotation(
                    x=list(total_plus_one_irrep_df_difference['input'])[i],
                    y=math.log10(list(total_plus_one_irrep_df_difference['difference'])[i]),
                    xref="x",
                    yref="y",
                    text=f"(n,t)={list(total_plus_one_irrep_df_difference['input'])[i]}, Diff={int(list(total_plus_one_irrep_df_difference['difference'])[i])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=15,
                    ay=-55,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#38638f",
                    opacity=0.6,
                    textangle=20
                    )
fig_dif.add_annotation(
                    x=list(total_plus_one_irrep_df_difference['input'])[18],
                    y=math.log10(list(total_plus_one_irrep_df_difference['difference'])[18]),
                    xref="x",
                    yref="y",
                    text=f"(n,t)={list(total_plus_one_irrep_df_difference['input'])[18]}, Diff={int(list(total_plus_one_irrep_df_difference['difference'])[18])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=-15,
                    ay=-55,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#38638f",
                    opacity=0.6,
                    textangle=20
                    )
fig_dif.add_annotation(
                    x=list(total_plus_one_irrep_df_difference['input'])[23],
                    y=math.log10(list(total_plus_one_irrep_df_difference['difference'])[23]),
                    xref="x",
                    yref="y",
                    text=f"(n,t)={list(total_plus_one_irrep_df_difference['input'])[23]}, Diff={int(list(total_plus_one_irrep_df_difference['difference'])[23])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=-15,
                    ay=-75,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#38638f",
                    opacity=0.6,
                    textangle=20
                    )
fig_dif.add_annotation(
                    x=list(total_plus_one_irrep_df_difference['input'])[25],
                    y=math.log10(list(total_plus_one_irrep_df_difference['difference'])[25]),
                    xref="x",
                    yref="y",
                    text=f"(n,t)={list(total_plus_one_irrep_df_difference['input'])[25]}, Diff={int(list(total_plus_one_irrep_df_difference['difference'])[25])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=-15,
                    ay=-75,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#38638f",
                    opacity=0.6,
                    textangle=20
                    )

ignore_if = 0
for i in range(1,38):
    if i!=30:
        if list(total_plus_one_irrep_df_difference['difference'])[i+1]>list(total_plus_one_irrep_df_difference['difference'])[i]<list(total_plus_one_irrep_df_difference['difference'])[i-1]:
            ignore_if +=1
            if ignore_if%2 == 1:
                fig_dif.add_annotation(
                        x=list(total_plus_one_irrep_df_difference['input'])[i],
                        y=math.log10(list(total_plus_one_irrep_df_difference['difference'])[i]),
                        xref="x",
                        yref="y",
                        text=f"(n,t)={list(total_plus_one_irrep_df_difference['input'])[i]}, Diff={int(list(total_plus_one_irrep_df_difference['difference'])[i])}",
                        showarrow=True,
                        font=dict(
                            family="Courier New, monospace",
                            size=10,
                            color="#ffffff"
                            ),
                        align="center",
                        arrowhead=2,
                        arrowsize=1,
                        arrowwidth=2,
                        arrowcolor="#636363",
                        ax=-15,
                        ay=55+i,
                        bordercolor="#c7c7c7",
                        borderwidth=2,
                        borderpad=4,
                        bgcolor="#584285",
                        opacity=0.6,
                        textangle=20
                        )

fig_dif.add_annotation(
                    x=list(total_plus_one_irrep_df_difference['input'])[30],
                    y=math.log10(list(total_plus_one_irrep_df_difference['difference'])[30]),
                    xref="x",
                    yref="y",
                    text=f"(n,t)={list(total_plus_one_irrep_df_difference['input'])[30]}, Diff={int(list(total_plus_one_irrep_df_difference['difference'])[30])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=15,
                    ay=35,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#584285",
                    opacity=0.6,
                    textangle=20
                    )
                
fig_dif.show()

In [ ]:
total_plus_one_irrep_df_difference.shape[0]

### Total Classification of Indicators when t=2

In [ ]:
Total_irreps_t_is_2_data = []
for n in range(1,151):
    if n%2==0:
        My_list = Final_Indicator_list_for_permutations_when_t_is_2(n)
        Computation = total_indicators_for_irreps_dim_t_given_n_as_list_plus_minus_zero(n,2)
        print(f'working on ({n})...')
        print('calculating...')
        Total_irreps_t_is_2_data.append([
            'My Formula +1',
            f'({n})',
            My_list[0]])
        Total_irreps_t_is_2_data.append([
            'My Formula -1',
            f'({n})',
            My_list[1]])
        Total_irreps_t_is_2_data.append([
            'My Formula 0',
            f'({n})',
            My_list[2]])    
        Total_irreps_t_is_2_data.append([
            'Direct Computation +1',
            f'({n})',
            Computation[0]])
        Total_irreps_t_is_2_data.append([
            'Direct Computation -1',
            f'({n})',
            Computation[1]])
        Total_irreps_t_is_2_data.append([
            'Direct Computation 0',
            f'({n})',
            Computation[2]])

In [ ]:
total_irreps_t_is_2_df = pd.DataFrame(Total_irreps_t_is_2_data, columns = ['func', 'input', 'result'])
#total_irreps_t_is_2_df = total_irreps_t_is_2_df[~total_plus_one_irrep_df['input'].isin(list(total_plus_one_irrep_df[total_plus_one_irrep_df['result']==0]['input']))]
total_irreps_t_is_2_df

In [ ]:
total_irreps_plus_one_t_is_2_df = total_irreps_t_is_2_df[(total_irreps_t_is_2_df['func'] == 'Direct Computation +1') | (total_irreps_t_is_2_df['func'] == 'My Formula +1')]
total_irreps_plus_one_t_is_2_df = total_irreps_plus_one_t_is_2_df[~total_irreps_plus_one_t_is_2_df['input'].isin(list(total_irreps_plus_one_t_is_2_df[total_irreps_plus_one_t_is_2_df['result']==0]['input']))]
total_irreps_plus_one_t_is_2_df

In [ ]:
total_irreps_plus_one_t_is_2_mine_df = total_irreps_plus_one_t_is_2_df[(total_irreps_plus_one_t_is_2_df['func'] == 'My Formula +1')]
total_irreps_plus_one_t_is_2_mine_df

In [ ]:
total_irreps_minus_one_t_is_2_df = total_irreps_t_is_2_df[(total_irreps_t_is_2_df['func'] == 'Direct Computation -1') | (total_irreps_t_is_2_df['func'] == 'My Formula -1')]
total_irreps_minus_one_t_is_2_df = total_irreps_minus_one_t_is_2_df[~total_irreps_minus_one_t_is_2_df['input'].isin(list(total_irreps_minus_one_t_is_2_df[total_irreps_minus_one_t_is_2_df['result']==0]['input']))]
total_irreps_minus_one_t_is_2_df

In [ ]:
total_irreps_minus_one_t_is_2_mine_df = total_irreps_minus_one_t_is_2_df[(total_irreps_minus_one_t_is_2_df['func'] == 'My Formula -1')]
total_irreps_minus_one_t_is_2_mine_df

In [ ]:
total_irreps_zeros_t_is_2_df = total_irreps_t_is_2_df[(total_irreps_t_is_2_df['func'] == 'Direct Computation 0') | (total_irreps_t_is_2_df['func'] == 'My Formula 0')]
total_irreps_zeros_t_is_2_df = total_irreps_zeros_t_is_2_df[~total_irreps_zeros_t_is_2_df['input'].isin(list(total_irreps_zeros_t_is_2_df[total_irreps_zeros_t_is_2_df['result']==0]['input']))]
total_irreps_zeros_t_is_2_df

In [ ]:
total_irreps_zeros_t_is_2_mine_df = total_irreps_zeros_t_is_2_df[(total_irreps_zeros_t_is_2_df['func'] == 'My Formula 0')]
total_irreps_zeros_t_is_2_mine_df

In [ ]:
total_irreps_all_mine_t_is_2_df = total_irreps_t_is_2_df[(total_irreps_t_is_2_df['func'] == 'My Formula -1') | (total_irreps_t_is_2_df['func'] == 'My Formula +1') | (total_irreps_t_is_2_df['func'] == 'My Formula 0')]
total_irreps_all_mine_t_is_2_df

In [ ]:
total_irreps_all_mine_not_zero_t_is_2_df = total_irreps_t_is_2_df[(total_irreps_t_is_2_df['func'] == 'My Formula -1') | (total_irreps_t_is_2_df['func'] == 'My Formula +1')]
total_irreps_all_mine_not_zero_t_is_2_df

In [ ]:
fig11 = px.line(total_irreps_plus_one_t_is_2_mine_df, x="input", y="result", labels={
                     "input": "(n)",
                     "result": "Total Number of irreps of Jn of dimension 2 with indicator +1",
                     },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of irreps of Jn of dimension 2 with indicator +1</sup>"
                       )
fig11.update_traces(patch={"line": {"color": "royalblue", "width": 2}})
# fig.update_traces(patch={"line": {"color": "blue", "width": 4, "dash": 'dot'}}, selector={"legendgroup": "Direct Computation"})

for i in range(1,72):
    if list(total_irreps_plus_one_t_is_2_mine_df['result'])[i+1]*1.2<list(total_irreps_plus_one_t_is_2_mine_df['result'])[i]>1.2*list(total_irreps_plus_one_t_is_2_mine_df['result'])[i-1]:
        if i!=21:
            fig11.add_annotation(
                    x=list(total_irreps_plus_one_t_is_2_mine_df['input'])[i],
                    y=list(total_irreps_plus_one_t_is_2_mine_df['result'])[i],
                    xref="x",
                    yref="y",
                    text=f"(n)={list(total_irreps_plus_one_t_is_2_mine_df['input'])[i]}, Total={int(list(total_irreps_plus_one_t_is_2_mine_df['result'])[i])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=-15,
                    ay=-15-i,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#185ba8",
                    opacity=0.6,
                    textangle=-12
                    )

fig11.add_annotation(
        x=list(total_irreps_plus_one_t_is_2_mine_df['input'])[21],
        y=list(total_irreps_plus_one_t_is_2_mine_df['result'])[21],
        xref="x",
        yref="y",
        text=f"(n)={list(total_irreps_plus_one_t_is_2_mine_df['input'])[21]}, Total={int(list(total_irreps_plus_one_t_is_2_mine_df['result'])[21])}",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=10,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=-35,
        ay=-75,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#185ba8",
        opacity=0.6,
        textangle=-12
        )            
            
fig11.show()



In [ ]:
fig12 = px.line(total_irreps_minus_one_t_is_2_mine_df, x="input", y="result", labels={
                     "input": "(n)",
                     "result": "Total Number of irreps of Jn of dimension 2 with indicator -1",
                     },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of irreps of Jn of dimension 2 with indicator -1</sup>"
                       )
fig12.update_traces(patch={"line": {"color": "darkslateblue", "width": 2}})
# fig.update_traces(patch={"line": {"color": "blue", "width": 4, "dash": 'dot'}}, selector={"legendgroup": "Direct Computation"})

alternate = 0
for i in range(1,34):
    if list(total_irreps_minus_one_t_is_2_mine_df['result'])[i+1]*1<list(total_irreps_minus_one_t_is_2_mine_df['result'])[i]>1*list(total_irreps_minus_one_t_is_2_mine_df['result'])[i-1]:
        alternate+=1
        if i not in [9,15,18,21,25,30,32]:
            if alternate%2==0:
                fig12.add_annotation(
                        x=list(total_irreps_minus_one_t_is_2_mine_df['input'])[i],
                        y=list(total_irreps_minus_one_t_is_2_mine_df['result'])[i],
                        xref="x",
                        yref="y",
                        text=f"(n)={list(total_irreps_minus_one_t_is_2_mine_df['input'])[i]}, Total={int(list(total_irreps_minus_one_t_is_2_mine_df['result'])[i])}",
                        showarrow=True,
                        font=dict(
                            family="Courier New, monospace",
                            size=10,
                            color="#ffffff"
                            ),
                        align="center",
                        arrowhead=2,
                        arrowsize=1,
                        arrowwidth=2,
                        arrowcolor="#636363",
                        ax=-15,
                        ay=-25-i,
                        bordercolor="#c7c7c7",
                        borderwidth=2,
                        borderpad=4,
                        bgcolor="#3e18a8",
                        opacity=0.6,
                        textangle=-12
                        )
            else:
                fig12.add_annotation(
                    x=list(total_irreps_minus_one_t_is_2_mine_df['input'])[i],
                    y=list(total_irreps_minus_one_t_is_2_mine_df['result'])[i],
                    xref="x",
                    yref="y",
                    text=f"(n)={list(total_irreps_minus_one_t_is_2_mine_df['input'])[i]}, Total={int(list(total_irreps_minus_one_t_is_2_mine_df['result'])[i])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=-15,
                    ay=-55-i,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#3e18a8",
                    opacity=0.6,
                    textangle=-12
                    )

                
for i in [9,18]:             
    fig12.add_annotation(
            x=list(total_irreps_minus_one_t_is_2_mine_df['input'])[i],
            y=list(total_irreps_minus_one_t_is_2_mine_df['result'])[i],
            xref="x",
            yref="y",
            text=f"(n)={list(total_irreps_minus_one_t_is_2_mine_df['input'])[i]}, Total={int(list(total_irreps_minus_one_t_is_2_mine_df['result'])[i])}",
            showarrow=True,
            font=dict(
                family="Courier New, monospace",
                size=10,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=-15,
            ay=-75-i,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor="#3e18a8",
            opacity=0.6,
            textangle=-12
            )
                
for i in [15,30]:             
    fig12.add_annotation(
            x=list(total_irreps_minus_one_t_is_2_mine_df['input'])[i],
            y=list(total_irreps_minus_one_t_is_2_mine_df['result'])[i],
            xref="x",
            yref="y",
            text=f"(n)={list(total_irreps_minus_one_t_is_2_mine_df['input'])[i]}, Total={int(list(total_irreps_minus_one_t_is_2_mine_df['result'])[i])}",
            showarrow=True,
            font=dict(
                family="Courier New, monospace",
                size=10,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=35,
            ay=-95-i,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor="#3e18a8",
            opacity=0.6,
            textangle=-12
            )
                

for i in [32]:             
    fig12.add_annotation(
            x=list(total_irreps_minus_one_t_is_2_mine_df['input'])[i],
            y=list(total_irreps_minus_one_t_is_2_mine_df['result'])[i],
            xref="x",
            yref="y",
            text=f"(n)={list(total_irreps_minus_one_t_is_2_mine_df['input'])[i]}, Total={int(list(total_irreps_minus_one_t_is_2_mine_df['result'])[i])}",
            showarrow=True,
            font=dict(
                family="Courier New, monospace",
                size=10,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=15,
            ay=-135,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor="#3e18a8",
            opacity=0.6,
            textangle=-12
            )
    
fig12.add_annotation(
        x=list(total_irreps_minus_one_t_is_2_mine_df['input'])[21],
        y=list(total_irreps_minus_one_t_is_2_mine_df['result'])[21],
        xref="x",
        yref="y",
        text=f"(n)={list(total_irreps_minus_one_t_is_2_mine_df['input'])[21]}, Total={int(list(total_irreps_minus_one_t_is_2_mine_df['result'])[21])}",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=10,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=15,
        ay=-55,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#3e18a8",
        opacity=0.6,
        textangle=-12
        )

fig12.add_annotation(
            x=list(total_irreps_minus_one_t_is_2_mine_df['input'])[25],
            y=list(total_irreps_minus_one_t_is_2_mine_df['result'])[25],
            xref="x",
            yref="y",
            text=f"(n)={list(total_irreps_minus_one_t_is_2_mine_df['input'])[25]}, Total={int(list(total_irreps_minus_one_t_is_2_mine_df['result'])[25])}",
            showarrow=True,
            font=dict(
                family="Courier New, monospace",
                size=10,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=-45,
            ay=-115,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor="#3e18a8",
            opacity=0.6,
            textangle=-12
            )   
    
fig12.show()


In [ ]:
fig13 = px.line(total_irreps_zeros_t_is_2_mine_df, x="input", y="result",labels={
                     "input": "(n)",
                     "result": "Total Number of irreps of Jn of dimension 2 with indicator 0"
                        }
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of irreps of Jn of dimension 2 with indicator 0</sup>"
                        )
fig13.update_traces(patch={"line": {"color": "darkmagenta", "width": 2}})
# fig.update_traces(patch={"line": {"color": "blue", "width": 4, "dash": 'dot'}}, selector={"legendgroup": "Direct Computation"})


for i in range(26,72):
    if list(total_irreps_zeros_t_is_2_mine_df['result'])[i+1]*2.5<list(total_irreps_zeros_t_is_2_mine_df['result'])[i]>2.5*list(total_irreps_plus_one_t_is_2_mine_df['result'])[i-1]:
        fig13.add_annotation(
                x=list(total_irreps_zeros_t_is_2_mine_df['input'])[i],
                y=list(total_irreps_zeros_t_is_2_mine_df['result'])[i],
                xref="x",
                yref="y",
                text=f"(n)={list(total_irreps_zeros_t_is_2_mine_df['input'])[i]}, Total={int(list(total_irreps_zeros_t_is_2_mine_df['result'])[i])}",
                showarrow=True,
                font=dict(
                    family="Courier New, monospace",
                    size=10,
                    color="#ffffff"
                    ),
                align="center",
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor="#636363",
                ax=-15,
                ay=-15-i,
                bordercolor="#c7c7c7",
                borderwidth=2,
                borderpad=4,
                bgcolor="#9a18a8",
                opacity=0.6,
                textangle=-12
                )

fig13.add_annotation(
        x=list(total_irreps_zeros_t_is_2_mine_df['input'])[70],
        y=list(total_irreps_zeros_t_is_2_mine_df['result'])[70],
        xref="x",
        yref="y",
        text=f"(n)={list(total_irreps_zeros_t_is_2_mine_df['input'])[70]}, Total={int(list(total_irreps_zeros_t_is_2_mine_df['result'])[70])}",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=10,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=-15,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#9a18a8",
        opacity=0.6,
        textangle=-12
        )        
        
        
fig13.show()


In [ ]:
fig15 = px.line(total_irreps_all_mine_t_is_2_df, x="input", y="result", color='func', labels={
                     "input": "(n,t)",
                     "result": "Total Number of irreps of Jn of dimension t with given Indicator",
                     "func": "Indicator Value:"},
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of irreps of Jn of dimension t with indicator +1</sup>"
                      )
fig15.update_traces(patch={"line": {"color": "royalblue", "width": 2}}, selector={"legendgroup": "My Formula +1"})
fig15.update_traces(patch={"line": {"color": "darkslateblue", "width": 2}}, selector={"legendgroup": "My Formula -1"})
fig15.update_traces(patch={"line": {"color": "darkmagenta", "width": 2}}, selector={"legendgroup": "My Formula 0"})


for i in range(26,72):
    if list(total_irreps_zeros_t_is_2_mine_df['result'])[i+1]*2.5<list(total_irreps_zeros_t_is_2_mine_df['result'])[i]>2.5*list(total_irreps_plus_one_t_is_2_mine_df['result'])[i-1]:
        fig15.add_annotation(
                x=list(total_irreps_zeros_t_is_2_mine_df['input'])[i],
                y=list(total_irreps_zeros_t_is_2_mine_df['result'])[i],
                xref="x",
                yref="y",
                text=f"(n)={list(total_irreps_zeros_t_is_2_mine_df['input'])[i]}, Total={int(list(total_irreps_zeros_t_is_2_mine_df['result'])[i])}",
                showarrow=True,
                font=dict(
                    family="Courier New, monospace",
                    size=10,
                    color="#ffffff"
                    ),
                align="center",
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor="#636363",
                ax=-15,
                ay=-15-i,
                bordercolor="#c7c7c7",
                borderwidth=2,
                borderpad=4,
                bgcolor="#9a18a8",
                opacity=0.6,
                textangle=-12
                )

fig15.add_annotation(
        x=list(total_irreps_zeros_t_is_2_mine_df['input'])[70],
        y=list(total_irreps_zeros_t_is_2_mine_df['result'])[70],
        xref="x",
        yref="y",
        text=f"(n)={list(total_irreps_zeros_t_is_2_mine_df['input'])[70]}, Total={int(list(total_irreps_zeros_t_is_2_mine_df['result'])[70])}",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=10,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=-15,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#9a18a8",
        opacity=0.6,
        textangle=-12
        )       
            
fig15.show()



In [ ]:
fig16 = px.line(total_irreps_all_mine_not_zero_t_is_2_df, x="input", y="result", color='func', labels={
                     "input": "(n,t)",
                     "result": "Total Number of irreps of Jn of dimension t with given Indicator",
                     "func": "Indicator Value:"},
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of irreps of Jn of dimension t with indicator +1</sup>"
                      )
fig16.update_traces(patch={"line": {"color": "royalblue", "width": 2}}, selector={"legendgroup": "My Formula +1"})
fig16.update_traces(patch={"line": {"color": "darkslateblue", "width": 2}}, selector={"legendgroup": "My Formula -1"})

for i in range(1,72):
    if list(total_irreps_plus_one_t_is_2_mine_df['result'])[i+1]*1.2<list(total_irreps_plus_one_t_is_2_mine_df['result'])[i]>1.2*list(total_irreps_plus_one_t_is_2_mine_df['result'])[i-1]:
        if i!=21:
            fig16.add_annotation(
                    x=list(total_irreps_plus_one_t_is_2_mine_df['input'])[i],
                    y=list(total_irreps_plus_one_t_is_2_mine_df['result'])[i],
                    xref="x",
                    yref="y",
                    text=f"(n)={list(total_irreps_plus_one_t_is_2_mine_df['input'])[i]}, Total={int(list(total_irreps_plus_one_t_is_2_mine_df['result'])[i])}",
                    showarrow=True,
                    font=dict(
                        family="Courier New, monospace",
                        size=10,
                        color="#ffffff"
                        ),
                    align="center",
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=-15,
                    ay=-15-i,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="#185ba8",
                    opacity=0.6,
                    textangle=-12
                    )

fig16.add_annotation(
        x=list(total_irreps_plus_one_t_is_2_mine_df['input'])[21],
        y=list(total_irreps_plus_one_t_is_2_mine_df['result'])[21],
        xref="x",
        yref="y",
        text=f"(n)={list(total_irreps_plus_one_t_is_2_mine_df['input'])[21]}, Total={int(list(total_irreps_plus_one_t_is_2_mine_df['result'])[21])}",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=10,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=-35,
        ay=-75,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#185ba8",
        opacity=0.6,
        textangle=-12
        )   

            
fig16.show()

In [ ]:
my_list6 = list(total_irreps_plus_one_t_is_2_df[total_irreps_plus_one_t_is_2_df['func']=='My Formula +1']['result'])
direct_computation6 = list(total_irreps_plus_one_t_is_2_df[total_irreps_plus_one_t_is_2_df['func']!='My Formula +1']['result'])

data_diff6 = {'My List Indicator +1':my_list6,
        'Direct Computation Indicator +1':direct_computation6}
 
# Create DataFrame
total_irreps_plus_one_t_is_2_df_difference = pd.DataFrame(data_diff6)

total_irreps_plus_one_t_is_2_df_difference['difference']=total_irreps_plus_one_t_is_2_df_difference['Direct Computation Indicator +1']-total_irreps_plus_one_t_is_2_df_difference['My List Indicator +1']
difference = list(total_irreps_plus_one_t_is_2_df_difference['difference'])
total_irreps_plus_one_t_is_2_df_difference['input'] = list(total_irreps_plus_one_t_is_2_df['input'].unique())
total_irreps_plus_one_t_is_2_df_difference[~total_irreps_plus_one_t_is_2_df_difference['input'].isin(list(total_irreps_plus_one_t_is_2_df_difference[total_irreps_plus_one_t_is_2_df_difference['difference']==0]['input']))]

In [ ]:
my_list7 = list(total_irreps_minus_one_t_is_2_df[total_irreps_minus_one_t_is_2_df['func']=='My Formula -1']['result'])
direct_computation7 = list(total_irreps_minus_one_t_is_2_df[total_irreps_minus_one_t_is_2_df['func']!='My Formula -1']['result'])

data_diff7 = {'My List Indicator -1':my_list7,
        'Direct Computation Indicator -1':direct_computation7}
 
# Create DataFrame
total_irreps_minus_one_t_is_2_df_difference = pd.DataFrame(data_diff7)

total_irreps_minus_one_t_is_2_df_difference['difference']=total_irreps_minus_one_t_is_2_df_difference['Direct Computation Indicator -1']-total_irreps_minus_one_t_is_2_df_difference['My List Indicator -1']
difference = list(total_irreps_minus_one_t_is_2_df_difference['difference'])
total_irreps_minus_one_t_is_2_df_difference['input'] = list(total_irreps_minus_one_t_is_2_df['input'].unique())
total_irreps_minus_one_t_is_2_df_difference[~total_irreps_minus_one_t_is_2_df_difference['input'].isin(list(total_irreps_minus_one_t_is_2_df_difference[total_irreps_minus_one_t_is_2_df_difference['difference']==0]['input']))]

In [ ]:
my_list8 = list(total_irreps_zeros_t_is_2_df[total_irreps_zeros_t_is_2_df['func']=='My Formula 0']['result'])
direct_computation8 = list(total_irreps_zeros_t_is_2_df[total_irreps_zeros_t_is_2_df['func']!='My Formula 0']['result'])

data_diff8 = {'My List Indicator 0':my_list8,
        'Direct Computation Indicator 0':direct_computation8}
 
# Create DataFrame
total_irreps_zeros_t_is_2_df_difference = pd.DataFrame(data_diff8)

total_irreps_zeros_t_is_2_df_difference['difference']=total_irreps_zeros_t_is_2_df_difference['Direct Computation Indicator 0']-total_irreps_zeros_t_is_2_df_difference['My List Indicator 0']
difference = list(total_irreps_zeros_t_is_2_df_difference['difference'])
total_irreps_zeros_t_is_2_df_difference['input'] = list(total_irreps_zeros_t_is_2_df['input'].unique())
total_irreps_zeros_t_is_2_df_difference[~total_irreps_zeros_t_is_2_df_difference['input'].isin(list(total_irreps_zeros_t_is_2_df_difference[total_irreps_zeros_t_is_2_df_difference['difference']==0]['input']))]

### Direct Computation of -1 indicators

In [ ]:
Total_minus_one_direct_comp = []
for n in range(1,51):
    for t in factors_of_n(n):
        if t%2==0:
            print(f'working on ({n},{t})...')
            if t<9:
                print('calculating...')
                Total_minus_one_direct_comp.append([
                    'Number Minus Ones',
                    f'({n},{t})',
                    total_indicators_for_irreps_dim_t_given_n_as_list_plus_minus_zero(n,t)[1]])
            else:
                print(f'{t} is too big, yuck!')

In [ ]:
total_minus_one_direct_df = pd.DataFrame(Total_minus_one_direct_comp, columns = ['func', 'input', 'result'])
total_minus_one_direct_df = total_minus_one_direct_df[~total_minus_one_direct_df['input'].isin(list(total_minus_one_direct_df[total_minus_one_direct_df['result']==0]['input']))]
total_minus_one_direct_df

In [ ]:
fig14 = px.line(total_minus_one_direct_df, x="input", y="result", log_y=True, labels={
                     "input": "(n,t)",
                     "result": "Computed Number of irreps of Jn of Dimension t with Indicator -1",
                     },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of orbits of length t containing r involutions</sup>"
              )
fig14.update_traces(patch={"line": {"color": "deeppink", "width": 2}})
# fig.update_traces(patch={"line": {"color": "blue", "width": 4, "dash": 'dot'}}, selector={"legendgroup": "Direct Computation"})


for i in range(1,16):
    if list(total_minus_one_direct_df['result'])[i+1]<list(total_minus_one_direct_df['result'])[i]>list(total_minus_one_direct_df['result'])[i-1]:
        fig14.add_annotation(
                x=list(total_minus_one_direct_df['input'])[i],
                y=math.log10(list(total_minus_one_direct_df['result'])[i]),
                xref="x",
                yref="y",
                text=f"(n,t)={list(total_minus_one_direct_df['input'])[i]}, Total={int(list(total_minus_one_direct_df['result'])[i])}",
                showarrow=True,
                font=dict(
                    family="Courier New, monospace",
                    size=10,
                    color="#ffffff"
                    ),
                align="center",
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor="#636363",
                ax=-15,
                ay=-20-i,
                bordercolor="#c7c7c7",
                borderwidth=2,
                borderpad=4,
                bgcolor="#a81859",
                opacity=0.6,
                textangle=-12
                )
        

for i in range(1,15):
    if list(total_minus_one_direct_df['result'])[i+1]>list(total_minus_one_direct_df['result'])[i]<list(total_minus_one_direct_df['result'])[i-1]:
        fig14.add_annotation(
                x=list(total_minus_one_direct_df['input'])[i],
                y=math.log10(list(total_minus_one_direct_df['result'])[i]),
                xref="x",
                yref="y",
                text=f"(n,t)={list(total_minus_one_direct_df['input'])[i]}, Total={int(list(total_minus_one_direct_df['result'])[i])}",
                showarrow=True,
                font=dict(
                    family="Courier New, monospace",
                    size=10,
                    color="#ffffff"
                    ),
                align="center",
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor="#636363",
                ax=-15,
                ay=30+i,
                bordercolor="#c7c7c7",
                borderwidth=2,
                borderpad=4,
                bgcolor="#a81859",
                opacity=0.6,
                textangle=12
                )


fig14.add_annotation(
        x=list(total_minus_one_direct_df['input'])[15],
        y=math.log10(list(total_minus_one_direct_df['result'])[15]),
        xref="x",
        yref="y",
        text=f"(n,t)={list(total_minus_one_direct_df['input'])[15]}, Total={int(list(total_minus_one_direct_df['result'])[15])}",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=10,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=-15,
        ay=20,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#a81859",
        opacity=0.6,
        textangle=12
        )       

fig14.show()


# Examining Ratio

In [ ]:
Ratio_data = []
for n in range(5,36):
    for t in factors_of_n(n):
        print(f'working on ({n},{t})...')
        if t<6:
            print('calculating...')
            Ratio_data.append([
                'Ratio of plus ones to total',
                f'({n},{t})',
                ratio_plus_one_indicator(n,t)])
            Ratio_data.append([
                'Ratio of minus ones to total',
                f'({n},{t})',
                ratio_minus_one_indicator(n,t)])
        else:
            print(f'{t} is too big, yuck!')

In [ ]:
ratio_df = pd.DataFrame(Ratio_data, columns = ['func', 'input', 'result'])
ratio_df = ratio_df[~ratio_df['input'].isin(list(ratio_df[ratio_df['result']==0]['input']))]
ratio_df

In [ ]:
ratio_plus_df = ratio_df[(ratio_df['func'] == 'Ratio of plus ones to total')]
ratio_plus_df

In [ ]:
ratio_minus_df = ratio_df[(ratio_df['func'] == 'Ratio of minus ones to total')]
ratio_minus_df

In [ ]:
fig20 = px.line(ratio_plus_df, x="input", y="result", labels={
                     "input": "(n,t)",
                     "result": "Percent of indicators of value +1 to total"
                      },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of permutations in symmetric group of order n-1 with stabilizer size n/t</sup>"
                      )
fig20.update_traces(patch={"line": {"color": "darkred", "width": 2}})

fig20.show()


In [ ]:
fig20 = px.line(ratio_minus_df, x="input", y="result", labels={
                     "input": "(n,t)",
                     "result": "Percent of indicators of value -1 to total"
                      },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of permutations in symmetric group of order n-1 with stabilizer size n/t</sup>"
                      )
fig20.update_traces(patch={"line": {"color": "darkred", "width": 2}})

fig20.show()

### Ratio t=2

In [ ]:
Ratio_t2_data = []
for n in range(5,200):
    if n%2==0:
        print(f'working on ({n})...')
        print('calculating...')
        Ratio_t2_data.append([
            'Ratio of plus ones to total',
            f'({n})',
            ratio_plus_one_indicator(n,2)])
        Ratio_t2_data.append([
            'Ratio of minus ones to total',
            f'({n})',
            ratio_minus_one_indicator(n,2)])

In [ ]:
ratio_t2_df = pd.DataFrame(Ratio_t2_data, columns = ['func', 'input', 'result'])
ratio_t2_df = ratio_t2_df[~ratio_t2_df['input'].isin(list(ratio_t2_df[ratio_t2_df['result']==0]['input']))]
ratio_t2_df

In [ ]:
ratio_t2_plus_df = ratio_t2_df[(ratio_t2_df['func'] == 'Ratio of plus ones to total')]
ratio_t2_plus_df

In [ ]:
ratio_t2_minus_df = ratio_t2_df[(ratio_t2_df['func'] == 'Ratio of minus ones to total')]
ratio_t2_minus_df

In [ ]:
fig21 = px.line(ratio_t2_plus_df, x="input", y="result", labels={
                     "input": "(n,t)",
                     "result": "Percent of indicators of value +1 to total"
                      },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of permutations in symmetric group of order n-1 with stabilizer size n/t</sup>"
                      )
fig21.update_traces(patch={"line": {"color": "darkred", "width": 2}})

fig21.show()

In [ ]:
fig22 = px.line(ratio_t2_minus_df, x="input", y="result", labels={
                     "input": "(n,t)",
                     "result": "Percent of indicators of value -1 to total"
                      },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of permutations in symmetric group of order n-1 with stabilizer size n/t</sup>"
                      )
fig22.update_traces(patch={"line": {"color": "darkred", "width": 2}})

fig22.show()

### Ratio t=3

In [ ]:
Ratio_t3_data = []
for n in range(5,400):
    if n%3==0:
        print(f'working on ({n})...')
        print('calculating...')
        Ratio_t3_data.append([
            'Ratio of plus ones to total',
            f'({n})',
            ratio_plus_one_indicator(n,3)])

In [ ]:
ratio_t3_df = pd.DataFrame(Ratio_t3_data, columns = ['func', 'input', 'result'])
ratio_t3_df = ratio_t3_df[~ratio_t3_df['input'].isin(list(ratio_t3_df[ratio_t3_df['result']==0]['input']))]
ratio_t3_df

In [ ]:
ratio_t3_plus_df = ratio_t3_df[(ratio_t3_df['func'] == 'Ratio of plus ones to total')]
ratio_t3_plus_df

In [ ]:
fig23 = px.line(ratio_t3_plus_df, x="input", y="result", labels={
                     "input": "(n,t)",
                     "result": "Percent of indicators of value +1 to total"
                      },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of permutations in symmetric group of order n-1 with stabilizer size n/t</sup>"
                      )
fig23.update_traces(patch={"line": {"color": "darkred", "width": 2}})

fig23.show()

### Ratio t=4

In [ ]:
Ratio_t4_data = []
for n in range(5,51):
    if n%4==0:
        print(f'working on ({n})...')
        print('calculating...')
        Ratio_t4_data.append([
            'Ratio of plus ones to total',
            f'({n})',
            ratio_plus_one_indicator(n,4)])
        Ratio_t4_data.append([
            'Ratio of minus ones to total',
            f'({n})',
            ratio_minus_one_indicator(n,4)])

In [ ]:
ratio_t4_df = pd.DataFrame(Ratio_t4_data, columns = ['func', 'input', 'result'])
ratio_t4_df = ratio_t4_df[~ratio_t4_df['input'].isin(list(ratio_t4_df[ratio_t4_df['result']==0]['input']))]
ratio_t4_df

In [ ]:
ratio_t4_plus_df = ratio_t4_df[(ratio_t4_df['func'] == 'Ratio of plus ones to total')]
ratio_t4_plus_df

In [ ]:
ratio_t4_minus_df = ratio_t4_df[(ratio_t4_df['func'] == 'Ratio of minus ones to total')]
ratio_t4_minus_df

In [ ]:
fig24 = px.line(ratio_t4_plus_df, x="input", y="result", labels={
                     "input": "(n,t)",
                     "result": "Ratio of indicators of value +1 to total"
                      },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of permutations in symmetric group of order n-1 with stabilizer size n/t</sup>"
                      )
fig24.update_traces(patch={"line": {"color": "darkred", "width": 2}})

fig24.show()

In [ ]:
fig25 = px.line(ratio_t4_minus_df, x="input", y="result", labels={
                     "input": "(n,t)",
                     "result": "Ratio of indicators of value -1 to total"
                      },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of permutations in symmetric group of order n-1 with stabilizer size n/t</sup>"
                      )
fig25.update_traces(patch={"line": {"color": "darkred", "width": 2}})

fig25.show()

### Ratio t=5

In [ ]:
Ratio_t5_data = []
for n in range(5,3001):
    if n%5==0:
        print(f'working on ({n})...')
        print('calculating...')
        Ratio_t5_data.append([
            'Ratio of plus ones to total',
            f'({n})',
            ratio_plus_one_indicator(n,5)])

In [ ]:
ratio_t5_df = pd.DataFrame(Ratio_t5_data, columns = ['func', 'input', 'result'])
ratio_t5_df = ratio_t5_df[~ratio_t5_df['input'].isin(list(ratio_t5_df[ratio_t5_df['result']==0]['input']))]

In [ ]:
ratio_t5_plus_df = ratio_t5_df[(ratio_t5_df['func'] == 'Ratio of plus ones to total')]
ratio_t5_plus_df

In [ ]:
fig26 = px.line(ratio_t5_plus_df, x="input", y="result", labels={
                     "input": "(n,t)",
                     "result": "Ratio of indicators of value +1 to total"
                      },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of permutations in symmetric group of order n-1 with stabilizer size n/t</sup>"
                      )
fig26.update_traces(patch={"line": {"color": "darkred", "width": 2}})

fig26.show()

### Ratio t=6

In [ ]:
Ratio_t6_data = []
for n in range(6,51):
    if n%6==0:
        print(f'working on ({n})...')
        print('calculating...')
        Ratio_t6_data.append([
            'Ratio of plus ones to total',
            f'({n})',
            ratio_plus_one_indicator(n,6)])
        Ratio_t6_data.append([
            'Ratio of minus ones to total',
            f'({n})',
            ratio_minus_one_indicator(n,6)])

In [ ]:
ratio_t6_df = pd.DataFrame(Ratio_t6_data, columns = ['func', 'input', 'result'])
ratio_t6_df = ratio_t6_df[~ratio_t6_df['input'].isin(list(ratio_t6_df[ratio_t6_df['result']==0]['input']))]

In [ ]:
ratio_t6_plus_df = ratio_t6_df[(ratio_t6_df['func'] == 'Ratio of plus ones to total')]
ratio_t6_plus_df

In [ ]:
ratio_t6_minus_df = ratio_t6_df[(ratio_t6_df['func'] == 'Ratio of minus ones to total')]
ratio_t6_minus_df

In [ ]:
fig27 = px.line(ratio_t6_plus_df, x="input", y="result", labels={
                     "input": "(n,t)",
                     "result": "Ratio of indicators of value +1 to total"
                      },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of permutations in symmetric group of order n-1 with stabilizer size n/t</sup>"
                      )
fig27.update_traces(patch={"line": {"color": "darkred", "width": 2}})

fig27.show()

In [ ]:
fig28 = px.line(ratio_t6_minus_df, x="input", y="result", labels={
                     "input": "(n,t)",
                     "result": "Ratio of indicators of value -1 to total"
                      },
                    #title="Comparing my Formula to Direct Computation: <br><sup> Number of permutations in symmetric group of order n-1 with stabilizer size n/t</sup>"
                      )
fig28.update_traces(patch={"line": {"color": "darkred", "width": 2}})

fig28.show()